In [ ]:
import os
import sys
import json
import threading
from pathlib import Path
from datetime import datetime
import tkinter as tk
from tkinter import ttk, filedialog, messagebox, scrolledtext
from collections import Counter
import re
import customtkinter as ctk
import whisper
import sounddevice as sd
import soundfile as sf
import numpy as np
from datetime import datetime
from anthropic import Anthropic
from typing import Optional, Dict, Any, List, Tuple
import queue
import time
import subprocess
import tempfile

# Set appearance mode and color theme
ctk.set_appearance_mode("dark")
ctk.set_default_color_theme("blue")

class AudioAnalyzerApp(ctk.CTk):
    def __init__(self):
        super().__init__()
        
        self.title("Audio Transcription & Analysis Tool")
        self.geometry("1400x800")
        
        # Make window resizable with minimum size
        self.minsize(1200, 600)
        self.resizable(True, True)
        
        # Initialize variables
        self.audio_file_path = None
        self.transcribed_text = ""
        self.api_key = None
        self.whisper_model = None
        self.error_queue = queue.Queue()
        self.model_loading = False
        
        # History for processed results (prompt, output) pairs
        self.process_history: List[Tuple[str, str]] = []
        self.current_history_index = -1
        
        # Recording variables
        self.is_recording = False
        self.recording_data = []
        self.recording_samplerate = 44100
        self.recording_thread = None
        self.recording_start_time = None
        self.recorded_file_path = None
        self.recording_process = None
        
        # Time segment variables
        self.audio_duration = None
        self.last_transcription_segment = (None, None)
        
        # Network plot variables
        self.current_network_plot_path = None
        self.word2vec_model = None
        self.network_photo = None  # Store the photo reference
        
        # Configure grid weight for resizing
        self.grid_columnconfigure(0, weight=1)
        self.grid_rowconfigure(0, weight=1)
        
        # Create main container with proper scaling
        self.main_container = ctk.CTkFrame(self)
        self.main_container.grid(row=0, column=0, sticky="nsew", padx=10, pady=10)
        self.main_container.grid_columnconfigure(0, weight=0)  # Sidebar doesn't resize
        self.main_container.grid_columnconfigure(1, weight=3)  # Main content scales
        self.main_container.grid_rowconfigure(0, weight=1)
        
        # Create UI components
        self.create_sidebar()
        self.create_main_panel()
        
        # Bind resize event for network plot
        self.bind("<Configure>", self.on_window_resize)
        
        # Start checking for errors from background threads
        self.check_error_queue()
        
        # Initialize Whisper model in background
        self.load_whisper_model()

    def toggle_time_segment(self):
        """Toggle between full audio and time segment selection"""
        if self.use_full_audio_var.get():
            # Disable time inputs
            self.start_time_entry.configure(state="disabled")
            self.end_time_entry.configure(state="disabled")
            self.duration_info_label.configure(text="Duration: Full audio")
        else:
            # Enable time inputs
            self.start_time_entry.configure(state="normal")
            self.end_time_entry.configure(state="normal")
            self.update_duration_info()
        
        # Update button states since we may need to re-transcribe
        self.update_button_states()
    
    def update_duration_info(self):
        """Update the duration info label based on selected time segment"""
        if self.use_full_audio_var.get():
            self.duration_info_label.configure(text="Duration: Full audio")
        else:
            try:
                start = float(self.start_time_var.get() or 0)
                end = float(self.end_time_var.get() or 0)
                
                if end > start:
                    duration = end - start
                    mins, secs = divmod(int(duration), 60)
                    self.duration_info_label.configure(
                        text=f"Duration: {mins:02d}:{secs:02d} ({duration:.1f}s)",
                        text_color=("green", "lightgreen")
                    )
                else:
                    self.duration_info_label.configure(
                        text="Invalid range: End must be after Start",
                        text_color=("red", "lightcoral")
                    )
            except ValueError:
                self.duration_info_label.configure(
                    text="Invalid input: Enter numbers only",
                    text_color=("red", "lightcoral")
                )
    
    def get_audio_duration(self, file_path):
        """Get the duration of an audio/video file in seconds"""
        try:
            # Try with soundfile first
            import soundfile as sf
            info = sf.info(file_path)
            return info.duration
        except:
            try:
                # Try with ffmpeg as fallback
                import subprocess
                import json
                
                cmd = [
                    'ffprobe', '-v', 'quiet',
                    '-print_format', 'json',
                    '-show_format',
                    file_path
                ]
                
                result = subprocess.run(cmd, capture_output=True, text=True)
                if result.returncode == 0:
                    data = json.loads(result.stdout)
                    duration = float(data['format']['duration'])
                    return duration
            except:
                pass
        
        # If all methods fail, return None
        return None
    
    def update_valid_range(self):
        """Update the valid range label based on loaded audio file"""
        if self.audio_file_path:
            duration = self.get_audio_duration(self.audio_file_path)
            if duration:
                mins, secs = divmod(int(duration), 60)
                self.valid_range_label.configure(
                    text=f"Valid range: 0 - {duration:.1f}s ({mins:02d}:{secs:02d})",
                    text_color=("blue", "lightblue")
                )
                
                # Update end time to match duration if using full audio
                if self.use_full_audio_var.get():
                    self.end_time_var.set(str(int(duration)))
                
                # Store duration for validation
                self.audio_duration = duration
            else:
                self.valid_range_label.configure(
                    text="Valid range: Unable to determine",
                    text_color=("orange", "darkorange")
                )
                self.audio_duration = None
        else:
            self.valid_range_label.configure(
                text="Valid range: No file loaded",
                text_color=("gray50", "gray50")
            )
            self.audio_duration = None
    
    def validate_time_segment(self):
        """Validate the selected time segment"""
        if self.use_full_audio_var.get():
            return True
        
        try:
            start = float(self.start_time_var.get() or 0)
            end = float(self.end_time_var.get() or 0)
            
            if start < 0:
                messagebox.showwarning("Invalid Time", "Start time cannot be negative")
                return False
            
            if end <= start:
                messagebox.showwarning("Invalid Time", "End time must be after start time")
                return False
            
            if hasattr(self, 'audio_duration') and self.audio_duration:
                if end > self.audio_duration:
                    messagebox.showwarning("Invalid Time", f"End time exceeds audio duration ({self.audio_duration:.1f}s)")
                    return False
            
            return True
            
        except ValueError:
            messagebox.showwarning("Invalid Input", "Please enter valid numbers for time segment")
            return False
    
    def get_time_segment_params(self):
        """Get the time segment parameters for transcription"""
        if self.use_full_audio_var.get():
            return None, None
        
        try:
            start = float(self.start_time_var.get() or 0)
            end = float(self.end_time_var.get() or 0)
            return start, end
        except ValueError:
            return None, None

    def create_sidebar(self):
        """Updated create_sidebar method with new features integrated"""
        # Create a scrollable sidebar container
        sidebar_container = ctk.CTkFrame(self.main_container, width=320)
        sidebar_container.grid(row=0, column=0, sticky="nsew", padx=(0, 10))
        sidebar_container.grid_propagate(False)
        sidebar_container.grid_rowconfigure(0, weight=1)
        sidebar_container.grid_columnconfigure(0, weight=1)
        
        # Create scrollable frame
        self.sidebar_scroll = ctk.CTkScrollableFrame(
            sidebar_container,
            width=300,
            corner_radius=0
        )
        self.sidebar_scroll.grid(row=0, column=0, sticky="nsew")
        
        # Use self.sidebar_scroll as the parent for all sidebar content
        self.sidebar = self.sidebar_scroll  # For compatibility with existing code
        
        # Title
        title_label = ctk.CTkLabel(
            self.sidebar, 
            text="DeScribe.AI", 
            font=ctk.CTkFont(size=24, weight="bold")
        )
        title_label.pack(pady=(20, 20))
        
        # Step 1: File Selection Section
        step1_frame = ctk.CTkFrame(self.sidebar, fg_color="transparent")
        step1_frame.pack(fill="x", padx=20, pady=(0, 15))
        
        ctk.CTkLabel(
            step1_frame, 
            text="Step 1: Select Audio File",
            font=ctk.CTkFont(size=14, weight="bold")
        ).pack(anchor="w", pady=(0, 10))
        
        self.file_button = ctk.CTkButton(
            step1_frame,
            text="📁 Choose Audio File",
            command=self.select_audio_file,
            height=40
        )
        self.file_button.pack(fill="x", pady=(0, 5))
        
        self.file_label = ctk.CTkLabel(
            step1_frame, 
            text="No file selected", 
            wraplength=250,
            font=ctk.CTkFont(size=11)
        )
        self.file_label.pack(anchor="w")
        
        # Recording Section (between Step 1 and Step 2)
        recording_frame = ctk.CTkFrame(self.sidebar, fg_color="transparent")
        recording_frame.pack(fill="x", padx=20, pady=(0, 15))
        
        ctk.CTkLabel(
            recording_frame,
            text="Or Record Audio:",
            font=ctk.CTkFont(size=14, weight="bold")
        ).pack(anchor="w", pady=(0, 10))
        
        # Recording controls frame
        record_controls = ctk.CTkFrame(recording_frame)
        record_controls.pack(fill="x")
        
        self.record_button = ctk.CTkButton(
            record_controls,
            text="🎤 Start Recording",
            command=self.toggle_recording,
            height=40,
            fg_color=("gray75", "gray25")
        )
        self.record_button.pack(fill="x", pady=(0, 5))
        
        # Recording status
        self.recording_status = ctk.CTkLabel(
            record_controls,
            text="Ready to record",
            font=ctk.CTkFont(size=11)
        )
        self.recording_status.pack(anchor="w", pady=(0, 5))
        
        # Recording timer
        self.recording_timer = ctk.CTkLabel(
            record_controls,
            text="",
            font=ctk.CTkFont(size=11),
            text_color=("red", "lightcoral")
        )
        self.recording_timer.pack(anchor="w")
        
        # Use recording button
        self.use_recording_button = ctk.CTkButton(
            record_controls,
            text="📂 Use Recording",
            command=self.use_recording,
            height=32,
            state="disabled"
        )
        self.use_recording_button.pack(fill="x", pady=(5, 0))
        
        # Time Segment Selection
        time_segment_frame = ctk.CTkFrame(self.sidebar, fg_color="transparent")
        time_segment_frame.pack(fill="x", padx=20, pady=(0, 15))
        
        ctk.CTkLabel(
            time_segment_frame,
            text="Time Segment Selection:",
            font=ctk.CTkFont(size=14, weight="bold")
        ).pack(anchor="w", pady=(0, 10))
        
        # Use full audio checkbox
        self.use_full_audio_var = ctk.BooleanVar(value=True)
        self.use_full_audio_check = ctk.CTkCheckBox(
            time_segment_frame,
            text="Use full audio",
            variable=self.use_full_audio_var,
            command=self.toggle_time_segment,
            font=ctk.CTkFont(size=12)
        )
        self.use_full_audio_check.pack(anchor="w", pady=(0, 10))
        
        # Time range inputs
        time_input_frame = ctk.CTkFrame(time_segment_frame)
        time_input_frame.pack(fill="x")
        
        # Start time
        start_frame = ctk.CTkFrame(time_input_frame)
        start_frame.pack(fill="x", pady=(0, 5))
        
        ctk.CTkLabel(start_frame, text="Start (seconds):", font=ctk.CTkFont(size=11)).pack(side="left", padx=(0, 10))
        self.start_time_var = ctk.StringVar(value="0")
        self.start_time_entry = ctk.CTkEntry(
            start_frame,
            width=80,
            textvariable=self.start_time_var,
            state="disabled"
        )
        self.start_time_entry.pack(side="left")
        
        # End time
        end_frame = ctk.CTkFrame(time_input_frame)
        end_frame.pack(fill="x", pady=(0, 5))
        
        ctk.CTkLabel(end_frame, text="End (seconds):", font=ctk.CTkFont(size=11)).pack(side="left", padx=(0, 10))
        self.end_time_var = ctk.StringVar(value="0")
        self.end_time_entry = ctk.CTkEntry(
            end_frame,
            width=80,
            textvariable=self.end_time_var,
            state="disabled"
        )
        self.end_time_entry.pack(side="left")
        
        # Duration info label
        self.duration_info_label = ctk.CTkLabel(
            time_segment_frame,
            text="Duration: Full audio",
            font=ctk.CTkFont(size=11),
            text_color=("gray50", "gray50")
        )
        self.duration_info_label.pack(anchor="w", pady=(5, 0))
        
        # Valid range label
        self.valid_range_label = ctk.CTkLabel(
            time_segment_frame,
            text="Valid range: No file loaded",
            font=ctk.CTkFont(size=11),
            text_color=("blue", "lightblue")
        )
        self.valid_range_label.pack(anchor="w", pady=(2, 0))
        
        # Step 2: Transcription Section
        step2_frame = ctk.CTkFrame(self.sidebar, fg_color="transparent")
        step2_frame.pack(fill="x", padx=20, pady=(0, 15))
        
        ctk.CTkLabel(
            step2_frame,
            text="Step 2: Transcribe Audio",
            font=ctk.CTkFont(size=14, weight="bold")
        ).pack(anchor="w", pady=(0, 10))
        
        # Whisper Model Selection
        model_container = ctk.CTkFrame(step2_frame)
        model_container.pack(fill="x", pady=(0, 10))
        
        ctk.CTkLabel(model_container, text="Whisper Model:", font=ctk.CTkFont(size=12)).pack(anchor="w", pady=(5, 5))
        
        model_select_frame = ctk.CTkFrame(model_container)
        model_select_frame.pack(fill="x")
        
        self.model_var = ctk.StringVar(value="base")
        models = ["tiny", "base", "small", "medium", "large"]
        self.model_menu = ctk.CTkOptionMenu(
            model_select_frame,
            values=models,
            variable=self.model_var,
            command=self.on_model_change,
            width=140
        )
        self.model_menu.pack(side="left", pady=(0, 5))
        
        # Model status indicator inline
        self.model_status = ctk.CTkLabel(
            model_select_frame, 
            text="⚪ Not loaded",
            font=ctk.CTkFont(size=11)
        )
        self.model_status.pack(side="left", padx=(10, 0))
        
        # Transcribe Button
        self.transcribe_button = ctk.CTkButton(
            step2_frame,
            text="🎙️ Transcribe",
            command=self.start_transcription,
            height=40,
            font=ctk.CTkFont(size=13, weight="bold"),
            state="disabled",
            fg_color=("gray75", "gray25")
        )
        self.transcribe_button.pack(fill="x", pady=(5, 0))
        
        # Clean Text Button
        self.clean_button = ctk.CTkButton(
            step2_frame,
            text="🧹 Clean Text",
            command=self.clean_transcribed_text,
            height=35,
            state="disabled",
            fg_color=("gray75", "gray25")
        )
        self.clean_button.pack(fill="x", pady=(5, 0))
        
        # Network Plot Button
        self.network_button = ctk.CTkButton(
            step2_frame,
            text="🕸️ Generate Network Plot",
            command=self.create_network_plot,
            height=35,
            state="disabled",
            fg_color=("gray75", "gray25")
        )
        self.network_button.pack(fill="x", pady=(5, 0))
        
        # Step 3: Processing Section
        step3_frame = ctk.CTkFrame(self.sidebar, fg_color="transparent")
        step3_frame.pack(fill="x", padx=20, pady=(0, 15))
        
        ctk.CTkLabel(
            step3_frame,
            text="Step 3: Process with AI",
            font=ctk.CTkFont(size=14, weight="bold")
        ).pack(anchor="w", pady=(0, 10))
        
        # API Key Entry
        api_container = ctk.CTkFrame(step3_frame)
        api_container.pack(fill="x", pady=(0, 10))
        
        ctk.CTkLabel(api_container, text="Anthropic API Key:", font=ctk.CTkFont(size=12)).pack(anchor="w", pady=(5, 5))
        
        api_entry_frame = ctk.CTkFrame(api_container)
        api_entry_frame.pack(fill="x")
        
        self.api_key_entry = ctk.CTkEntry(api_entry_frame, show="*", width=180)
        self.api_key_entry.pack(side="left", pady=(0, 5))
        
        self.save_api_button = ctk.CTkButton(
            api_entry_frame,
            text="Save",
            command=self.save_api_key,
            width=60,
            height=28
        )
        self.save_api_button.pack(side="left", padx=(5, 0))
        
        # Create the new summarize section with templates and tones
        self.create_summarize_section(step3_frame)
        
        # Create the translation section
        self.create_translation_section(self.sidebar)
        
        # Output Format
        format_frame = ctk.CTkFrame(step3_frame)
        format_frame.pack(fill="x", pady=(10, 0))
        
        ctk.CTkLabel(format_frame, text="Output Format:", font=ctk.CTkFont(size=11)).pack(side="left", padx=(0, 10))
        
        self.output_format = ctk.StringVar(value="Markdown")
        formats = ["Markdown", "Plain Text", "JSON", "Bullet Points", "LaTeX PDF"]
        self.format_menu = ctk.CTkOptionMenu(
            format_frame,
            values=formats,
            variable=self.output_format,
            width=140,
            height=28
        )
        self.format_menu.pack(side="left", fill="x", expand=True)
        
        # Progress Section
        progress_frame = ctk.CTkFrame(self.sidebar, fg_color="transparent")
        progress_frame.pack(fill="x", padx=20, pady=(20, 20))
        
        self.progress_bar = ctk.CTkProgressBar(progress_frame)
        self.progress_bar.pack(fill="x", pady=(0, 5))
        self.progress_bar.set(0)
        
        self.status_label = ctk.CTkLabel(
            progress_frame, 
            text="Ready", 
            font=ctk.CTkFont(size=11)
        )
        self.status_label.pack()

    def create_summarize_section(self, parent_frame):
        """Create the summarize section with templates and tones"""
        # Summarize Options
        summarize_frame = ctk.CTkFrame(parent_frame)
        summarize_frame.pack(fill="x", pady=(10, 0))
        
        # Template selection
        template_frame = ctk.CTkFrame(summarize_frame)
        template_frame.pack(fill="x", pady=(0, 10))
        
        ctk.CTkLabel(template_frame, text="Template:", font=ctk.CTkFont(size=12)).pack(side="left", padx=(0, 10))
        
        self.summary_template_var = ctk.StringVar(value="Standard")
        templates = [
            "Standard",
            "Executive Summary",
            "Academic Paper",
            "Meeting Minutes",
            "Podcast Notes",
            "Lecture Notes",
            "Interview Summary",
            "Technical Report",
            "News Article",
            "Research Brief"
        ]
        self.template_menu = ctk.CTkOptionMenu(
            template_frame,
            values=templates,
            variable=self.summary_template_var,
            width=140,
            height=28
        )
        self.template_menu.pack(side="left", fill="x", expand=True)
        
        # Tone selection
        tone_frame = ctk.CTkFrame(summarize_frame)
        tone_frame.pack(fill="x", pady=(0, 10))
        
        ctk.CTkLabel(tone_frame, text="Tone:", font=ctk.CTkFont(size=12)).pack(side="left", padx=(0, 10))
        
        self.summary_tone_var = ctk.StringVar(value="Professional")
        tones = [
            "Professional",
            "Casual",
            "Academic",
            "Technical",
            "Creative",
            "Formal",
            "Conversational",
            "Analytical",
            "Persuasive",
            "Objective"
        ]
        self.tone_menu = ctk.CTkOptionMenu(
            tone_frame,
            values=tones,
            variable=self.summary_tone_var,
            width=140,
            height=28
        )
        self.tone_menu.pack(side="left", fill="x", expand=True)
        
        # Word limit setting
        word_limit_frame = ctk.CTkFrame(summarize_frame)
        word_limit_frame.pack(fill="x", pady=(0, 10))
        
        ctk.CTkLabel(word_limit_frame, text="Word Limit:", font=ctk.CTkFont(size=12)).pack(side="left", padx=(0, 10))
        
        self.word_limit_var = ctk.StringVar(value="500")
        self.word_limit_entry = ctk.CTkEntry(
            word_limit_frame,
            width=80,
            textvariable=self.word_limit_var,
            placeholder_text="500"
        )
        self.word_limit_entry.pack(side="left")
        
        ctk.CTkLabel(word_limit_frame, text="words", font=ctk.CTkFont(size=11)).pack(side="left", padx=(5, 0))
        
        # Summarize Button
        self.summarize_button = ctk.CTkButton(
            summarize_frame,
            text="📝 Summarize",
            command=self.summarize_transcription,
            height=40,
            state="disabled",
            font=ctk.CTkFont(size=13, weight="bold"),
            fg_color=("gray75", "gray25")
        )
        self.summarize_button.pack(fill="x", pady=(5, 0))
        
        return summarize_frame

    def create_translation_section(self, parent_frame):
        """Create the translation section in the sidebar"""
        # Translation Section (add after Summarize section)
        translation_frame = ctk.CTkFrame(parent_frame, fg_color="transparent")
        translation_frame.pack(fill="x", padx=20, pady=(15, 0))
        
        ctk.CTkLabel(
            translation_frame,
            text="Translation",
            font=ctk.CTkFont(size=14, weight="bold")
        ).pack(anchor="w", pady=(0, 10))
        
        # Language selection
        lang_select_frame = ctk.CTkFrame(translation_frame)
        lang_select_frame.pack(fill="x", pady=(0, 10))
        
        ctk.CTkLabel(lang_select_frame, text="Target Language:", font=ctk.CTkFont(size=12)).pack(anchor="w", pady=(0, 5))
        
        self.target_language_var = ctk.StringVar(value="Spanish")
        languages = [
            "Spanish", "French", "German", "Italian", "Portuguese", "Dutch",
            "Russian", "Chinese (Simplified)", "Chinese (Traditional)", 
            "Japanese", "Korean", "Arabic", "Hindi", "Bengali",
            "Turkish", "Polish", "Vietnamese", "Thai", "Indonesian",
            "Swedish", "Norwegian", "Danish", "Finnish", "Greek",
            "Hebrew", "Czech", "Hungarian", "Romanian", "Ukrainian"
        ]
        self.language_menu = ctk.CTkOptionMenu(
            lang_select_frame,
            values=languages,
            variable=self.target_language_var,
            width=180,
            height=32
        )
        self.language_menu.pack(fill="x")
        
        # Translation style options
        style_frame = ctk.CTkFrame(translation_frame)
        style_frame.pack(fill="x", pady=(0, 10))
        
        ctk.CTkLabel(style_frame, text="Translation Style:", font=ctk.CTkFont(size=12)).pack(anchor="w", pady=(0, 5))
        
        self.translation_style_var = ctk.StringVar(value="Natural")
        styles = ["Natural", "Literal", "Professional", "Colloquial", "Technical"]
        self.style_menu = ctk.CTkOptionMenu(
            style_frame,
            values=styles,
            variable=self.translation_style_var,
            width=180,
            height=28
        )
        self.style_menu.pack(fill="x")
        
        # Preserve formatting checkbox
        self.preserve_formatting_var = ctk.BooleanVar(value=True)
        self.preserve_formatting_check = ctk.CTkCheckBox(
            translation_frame,
            text="Preserve original formatting",
            variable=self.preserve_formatting_var,
            font=ctk.CTkFont(size=12)
        )
        self.preserve_formatting_check.pack(anchor="w", pady=(5, 10))
        
        # Translate button
        self.translate_button = ctk.CTkButton(
            translation_frame,
            text="🌐 Translate",
            command=self.translate_transcription,
            height=40,
            state="disabled",
            font=ctk.CTkFont(size=13, weight="bold"),
            fg_color=("gray75", "gray25")
        )
        self.translate_button.pack(fill="x")
        
        return translation_frame
    
    def translate_transcription(self):
        """Translate the transcribed text to the selected language"""
        if not self.transcribed_text:
            messagebox.showwarning("No Content", "No transcription available to translate")
            return
        
        api_key = self.api_key_entry.get() or self.api_key
        if not api_key:
            messagebox.showwarning("API Key Required", "Please enter your Anthropic API key")
            return
        
        target_language = self.target_language_var.get()
        translation_style = self.translation_style_var.get()
        preserve_formatting = self.preserve_formatting_var.get()
        
        def translate():
            try:
                self.after(0, lambda: self.translate_button.configure(state="disabled"))
                self.after(0, lambda: self.update_status(f"Translating to {target_language}..."))
                self.after(0, lambda: self.progress_bar.set(0.5))
                
                prompt = self.get_translation_prompt(
                    self.transcribed_text,
                    target_language,
                    translation_style,
                    preserve_formatting
                )
                
                result = self.process_with_claude_enhanced(self.transcribed_text, prompt)
                
                if result:
                    # Update the result with language info
                    result_with_header = f"[Translation to {target_language}]\n\n{result}"
                    self.after(0, lambda: self.update_result(prompt, result_with_header))
                    self.after(0, lambda: self.update_status(f"Translation to {target_language} completed"))
                else:
                    self.after(0, lambda: self.update_status("Translation failed"))
                
            except Exception as e:
                self.error_queue.put(("Translation Error", f"Failed to translate: {str(e)}"))
            finally:
                self.after(0, lambda: self.progress_bar.set(0))
                self.after(0, lambda: self.translate_button.configure(state="normal"))
        
        threading.Thread(target=translate, daemon=True).start()
    
    def get_translation_prompt(self, text: str, target_language: str, style: str, preserve_formatting: bool) -> str:
        """Generate translation prompt based on settings"""
        output_format = self.output_format.get()
        
        # Style-specific instructions
        style_instructions = {
            "Natural": "Translate in a natural, fluent way that sounds native to the target language.",
            "Literal": "Provide a more literal translation that stays close to the original structure.",
            "Professional": "Use formal, professional language suitable for business or academic contexts.",
            "Colloquial": "Use everyday, conversational language with appropriate idioms.",
            "Technical": "Maintain technical terminology and precision, transliterating terms when necessary."
        }
        
        # Format-specific instructions for different languages
        format_instructions = {
            "markdown": "Maintain Markdown formatting. Translate headers, lists, and content while preserving structure.",
            "plain text": "Provide plain text translation without special formatting.",
            "json": f"Return as JSON with structure: {{\"language\": \"{target_language}\", \"translation\": \"...\", \"notes\": \"...\"}}",
            "bullet points": "Maintain bullet point structure while translating content.",
            "latex pdf": "Preserve LaTeX commands and structure. Translate content while keeping LaTeX formatting intact. Use appropriate language packages if needed (e.g., \\usepackage[arabic]{babel} for Arabic)."
        }
        
        # Special handling for certain languages
        special_instructions = ""
        if target_language in ["Arabic", "Hebrew"]:
            special_instructions = "\nNote: This language uses right-to-left script. Ensure proper text direction in the output."
        elif target_language in ["Chinese (Simplified)", "Chinese (Traditional)", "Japanese", "Korean"]:
            special_instructions = "\nNote: Handle character encoding carefully for East Asian languages."
        
        prompt = f"""Translate the following transcribed text to {target_language}.
    
    TRANSLATION STYLE: {style}
    {style_instructions.get(style, '')}
    
    OUTPUT FORMAT: {output_format}
    {format_instructions.get(output_format.lower(), '')}
    
    {"PRESERVE ORIGINAL FORMATTING: Maintain paragraph breaks, punctuation style, and structure." if preserve_formatting else "ADAPT FORMATTING: Adjust formatting to be natural for the target language."}
    {special_instructions}
    
    Important instructions:
    1. Translate all content accurately while maintaining the original meaning
    2. Adapt idioms and expressions to be culturally appropriate
    3. For technical terms without direct translations, provide the translation followed by the original term in parentheses
    4. Ensure grammatical correctness in the target language
    5. If the output format is LaTeX, include appropriate language packages in the preamble
    
    Text to translate:
    {text}"""
        
        return prompt
    
    def ensure_latex_format_multilingual(self, text: str, language: str) -> str:
        """Ensure LaTeX format works with multiple languages"""
        
        # Language-specific LaTeX packages
        language_packages = {
            "Arabic": "\\usepackage[arabic]{babel}\n\\usepackage{arabtex}",
            "Chinese (Simplified)": "\\usepackage{CJKutf8}",
            "Chinese (Traditional)": "\\usepackage{CJKutf8}",
            "Japanese": "\\usepackage{CJKutf8}",
            "Korean": "\\usepackage{CJKutf8}",
            "Russian": "\\usepackage[russian]{babel}",
            "Greek": "\\usepackage[greek]{babel}",
            "Hebrew": "\\usepackage[hebrew]{babel}"
        }
        
        # First ensure basic LaTeX format
        text = self.ensure_latex_format(text)
        
        # Add language-specific packages if needed
        if language in language_packages:
            lines = text.split('\n')
            for i, line in enumerate(lines):
                if line.startswith("\\documentclass"):
                    # Insert language package after documentclass
                    package = language_packages[language]
                    lines.insert(i + 1, package)
                    break
            text = '\n'.join(lines)
        
        # Special handling for CJK languages
        if language in ["Chinese (Simplified)", "Chinese (Traditional)", "Japanese", "Korean"]:
            # Wrap content in CJK environment
            if "\\begin{document}" in text and "\\end{document}" in text:
                start = text.index("\\begin{document}") + len("\\begin{document}")
                end = text.index("\\end{document}")
                content = text[start:end]
                
                cjk_env = "CJK*{UTF8}{gbsn}" if "Chinese" in language else "CJK*{UTF8}{min}"
                wrapped_content = f"\n\\begin{{{cjk_env}}}\n{content}\n\\end{{CJK*}}\n"
                
                text = text[:start] + wrapped_content + text[end:]
        
        return text

    def create_main_panel(self):
        """Create main panel with three columns: left sidebar, center content, right visualization"""
        # Main Panel Container
        self.main_panel = ctk.CTkFrame(self.main_container)
        self.main_panel.grid(row=0, column=1, sticky="nsew")
        self.main_panel.grid_columnconfigure(0, weight=3)  # Center panel gets more weight
        self.main_panel.grid_columnconfigure(1, weight=2)  # Right panel for visualization
        self.main_panel.grid_rowconfigure(0, weight=1)
        
        # LEFT/CENTER: Original tabbed content
        self.center_container = ctk.CTkFrame(self.main_panel)
        self.center_container.grid(row=0, column=0, sticky="nsew", padx=(0, 5))
        self.center_container.grid_columnconfigure(0, weight=1)
        self.center_container.grid_rowconfigure(0, weight=1)
        self.center_container.grid_rowconfigure(1, weight=0)
        
        # Tab View (in center)
        self.tabview = ctk.CTkTabview(self.center_container)
        self.tabview.grid(row=0, column=0, sticky="nsew", padx=10, pady=10)
        
        # Transcription Tab
        self.trans_tab = self.tabview.add("📝 Transcription")
        trans_frame = ctk.CTkFrame(self.trans_tab)
        trans_frame.pack(fill="both", expand=True, padx=5, pady=5)
        
        # Transcription header with word count
        trans_header = ctk.CTkFrame(trans_frame, height=30)
        trans_header.pack(fill="x", pady=(0, 5))
        
        self.trans_word_count = ctk.CTkLabel(
            trans_header, 
            text="Words: 0 | Characters: 0",
            font=ctk.CTkFont(size=11)
        )
        self.trans_word_count.pack(side="left")
        
        self.trans_status = ctk.CTkLabel(
            trans_header,
            text="No transcription yet",
            font=ctk.CTkFont(size=11),
            text_color=("gray50", "gray50")
        )
        self.trans_status.pack(side="right")
        
        self.trans_text = ctk.CTkTextbox(
            trans_frame,
            font=ctk.CTkFont(size=13),
            wrap="word"
        )
        self.trans_text.pack(fill="both", expand=True)
        
        # Processed Result Tab - Split view
        self.result_tab = self.tabview.add("✨ Processed Result")
        result_container = ctk.CTkFrame(self.result_tab)
        result_container.pack(fill="both", expand=True, padx=5, pady=5)
        result_container.grid_columnconfigure(0, weight=1)
        result_container.grid_rowconfigure(0, weight=2)
        result_container.grid_rowconfigure(1, weight=1)
        
        # Upper section - API output
        upper_frame = ctk.CTkFrame(result_container)
        upper_frame.grid(row=0, column=0, sticky="nsew", pady=(0, 5))
        
        # Result header with navigation
        result_header = ctk.CTkFrame(upper_frame, height=30)
        result_header.pack(fill="x", pady=(0, 5))
        
        # Navigation buttons for history
        nav_frame = ctk.CTkFrame(result_header)
        nav_frame.pack(side="left")
        
        self.prev_button = ctk.CTkButton(
            nav_frame,
            text="◀",
            width=30,
            height=25,
            command=self.navigate_history_prev,
            state="disabled"
        )
        self.prev_button.pack(side="left", padx=2)
        
        self.history_label = ctk.CTkLabel(
            nav_frame,
            text="0/0",
            font=ctk.CTkFont(size=11),
            width=50
        )
        self.history_label.pack(side="left", padx=5)
        
        self.next_button = ctk.CTkButton(
            nav_frame,
            text="▶",
            width=30,
            height=25,
            command=self.navigate_history_next,
            state="disabled"
        )
        self.next_button.pack(side="left", padx=2)
        
        self.result_status = ctk.CTkLabel(
            result_header,
            text="No processed result yet",
            font=ctk.CTkFont(size=11),
            text_color=("gray50", "gray50")
        )
        self.result_status.pack(side="right")
        
        self.result_text = ctk.CTkTextbox(
            upper_frame,
            font=ctk.CTkFont(size=13),
            wrap="word"
        )
        self.result_text.pack(fill="both", expand=True)
        
        # Lower section - Custom prompt
        lower_frame = ctk.CTkFrame(result_container)
        lower_frame.grid(row=1, column=0, sticky="nsew", pady=(5, 0))
        
        # Custom prompt header
        prompt_header = ctk.CTkFrame(lower_frame)
        prompt_header.pack(fill="x", pady=(5, 5))
        
        ctk.CTkLabel(
            prompt_header,
            text="Custom Prompt (use {text} for transcription):",
            font=ctk.CTkFont(size=12)
        ).pack(side="left")
        
        # Custom prompt input
        prompt_input_frame = ctk.CTkFrame(lower_frame)
        prompt_input_frame.pack(fill="both", expand=True, pady=(0, 5))
        
        self.custom_prompt_text = ctk.CTkTextbox(
            prompt_input_frame,
            height=100,
            font=ctk.CTkFont(size=12)
        )
        self.custom_prompt_text.pack(fill="both", expand=True, pady=(0, 5))
        self.custom_prompt_text.insert("1.0", "Analyze the following text and provide insights:\n\n{text}")
        
        # Process custom prompt button
        self.process_custom_button = ctk.CTkButton(
            prompt_input_frame,
            text="▶ Process Custom Prompt",
            command=self.process_custom_prompt,
            height=32,
            state="disabled"
        )
        self.process_custom_button.pack(fill="x")
        
        # Bottom toolbar (for center panel)
        toolbar_frame = ctk.CTkFrame(self.center_container)
        toolbar_frame.grid(row=1, column=0, sticky="ew", padx=10, pady=(0, 10))
        
        # Left side buttons
        left_buttons = ctk.CTkFrame(toolbar_frame)
        left_buttons.pack(side="left")
        
        ctk.CTkButton(
            left_buttons,
            text="📋 Copy Current Tab",
            command=self.copy_to_clipboard,
            width=130,
            height=32
        ).pack(side="left", padx=2)
        
        ctk.CTkButton(
            left_buttons,
            text="💾 Export Transcription",
            command=lambda: self.export_text_enhanced("transcription"),
            width=140,
            height=32
        ).pack(side="left", padx=2)
        
        ctk.CTkButton(
            left_buttons,
            text="💾 Export Result",
            command=lambda: self.export_text_enhanced("result"),
            width=120,
            height=32
        ).pack(side="left", padx=2)
        
        # Right side buttons
        ctk.CTkButton(
            toolbar_frame,
            text="🗑️ Clear All",
            command=self.clear_all,
            width=100,
            height=32,
            fg_color=("gray75", "gray25")
        ).pack(side="right", padx=2)
        
        # RIGHT: Visualization Panel with proper scaling
        self.viz_panel = ctk.CTkFrame(self.main_panel)
        self.viz_panel.grid(row=0, column=1, sticky="nsew", padx=(5, 0))
        self.viz_panel.grid_columnconfigure(0, weight=1)
        self.viz_panel.grid_rowconfigure(0, weight=2)  # Network plot gets 2/3
        self.viz_panel.grid_rowconfigure(1, weight=1)  # Semantic summary gets 1/3
        
        # Upper 2/3: Network Plot Display
        self.network_frame = ctk.CTkFrame(self.viz_panel)
        self.network_frame.grid(row=0, column=0, sticky="nsew", padx=5, pady=(5, 2))
        self.network_frame.grid_columnconfigure(0, weight=1)
        self.network_frame.grid_rowconfigure(1, weight=1)  # Make display area expandable
        
        # Network plot header
        network_header = ctk.CTkFrame(self.network_frame, height=30)
        network_header.grid(row=0, column=0, sticky="ew", pady=(5, 5))
        
        ctk.CTkLabel(
            network_header,
            text="🕸️ Semantic Network",
            font=ctk.CTkFont(size=14, weight="bold")
        ).pack(side="left", padx=10)
        
        # Refresh button for network
        self.refresh_network_btn = ctk.CTkButton(
            network_header,
            text="↻ Regenerate",
            width=100,
            height=28,
            command=self.create_network_plot,
            state="disabled"
        )
        self.refresh_network_btn.pack(side="right", padx=10)
        
        # Network plot display area - make it expandable
        self.network_display = ctk.CTkLabel(
            self.network_frame,
            text="No network generated yet\n\nClick 'Generate Network Plot' to visualize",
            font=ctk.CTkFont(size=12),
            text_color=("gray50", "gray50")
        )
        self.network_display.grid(row=1, column=0, sticky="nsew", padx=10, pady=10)
        
        # Lower 1/3: Semantic Summary
        summary_frame = ctk.CTkFrame(self.viz_panel)
        summary_frame.grid(row=1, column=0, sticky="nsew", padx=5, pady=(2, 5))
        
        # Summary header
        summary_header = ctk.CTkFrame(summary_frame, height=30)
        summary_header.pack(fill="x", pady=(5, 5))
        
        ctk.CTkLabel(
            summary_header,
            text="📊 Semantic & Tone Summary",
            font=ctk.CTkFont(size=14, weight="bold")
        ).pack(side="left", padx=10)
        
        # Generate summary button
        self.generate_summary_btn = ctk.CTkButton(
            summary_header,
            text="✨ Generate",
            width=100,
            height=28,
            command=self.generate_semantic_summary,
            state="disabled"
        )
        self.generate_summary_btn.pack(side="right", padx=10)
        
        # Summary display
        self.semantic_summary_text = ctk.CTkTextbox(
            summary_frame,
            font=ctk.CTkFont(size=12),
            wrap="word",
            height=100
        )
        self.semantic_summary_text.pack(fill="both", expand=True, padx=10, pady=(0, 10))
        self.semantic_summary_text.insert("1.0", "No summary generated yet.\n\nClick 'Generate' to create a semantic & tone summary of the transcribed text.")
        self.semantic_summary_text.configure(state="disabled")

    def on_window_resize(self, event=None):
        """Handle window resize events to update network plot display"""
        # Only process resize events for the main window
        if event and event.widget != self:
            return
        
        # Check if we have a network plot to resize
        if hasattr(self, 'current_network_plot_path') and self.current_network_plot_path:
            if Path(self.current_network_plot_path).exists():
                # Delay the resize to avoid too many updates
                if hasattr(self, '_resize_after_id'):
                    self.after_cancel(self._resize_after_id)
                
                # Schedule resize after 100ms of no activity
                self._resize_after_id = self.after(100, lambda: self.resize_network_plot())
    
    def resize_network_plot(self):
        """Resize the network plot to fit the current panel size"""
        if not self.current_network_plot_path or not Path(self.current_network_plot_path).exists():
            return
        
        try:
            from PIL import Image, ImageTk
            
            # Get the current size of the network display area
            display_width = self.network_display.winfo_width()
            display_height = self.network_display.winfo_height()
            
            # Ensure minimum size
            if display_width < 100 or display_height < 100:
                return
            
            # Load the original image
            img = Image.open(self.current_network_plot_path)
            
            # Calculate scaling to maintain aspect ratio with padding
            padding = 20
            max_width = display_width - padding
            max_height = display_height - padding
            
            img_width, img_height = img.size
            scale = min(max_width/img_width, max_height/img_height)
            
            # Limit maximum scale to avoid pixelation
            scale = min(scale, 2.0)
            
            new_width = int(img_width * scale)
            new_height = int(img_height * scale)
            
            # Resize image with high quality
            img = img.resize((new_width, new_height), Image.Resampling.LANCZOS)
            
            # Convert to PhotoImage
            photo = ImageTk.PhotoImage(img)
            
            # Update the display
            self.network_display.configure(image=photo, text="")
            self.network_display.image = photo  # Keep a reference
            self.network_photo = photo  # Store reference
            
        except Exception as e:
            print(f"Error resizing network plot: {e}")
    
    def navigate_history_prev(self):
        """Navigate to previous result in history"""
        if self.current_history_index > 0:
            self.current_history_index -= 1
            self.display_history_item()
    
    def navigate_history_next(self):
        """Navigate to next result in history"""
        if self.current_history_index < len(self.process_history) - 1:
            self.current_history_index += 1
            self.display_history_item()
    
    def display_history_item(self):
        """Display the current history item"""
        if 0 <= self.current_history_index < len(self.process_history):
            prompt, result = self.process_history[self.current_history_index]
            
            # Update result text
            self.result_text.delete("1.0", "end")
            self.result_text.insert("1.0", result)
            
            # Update custom prompt text
            self.custom_prompt_text.delete("1.0", "end")
            self.custom_prompt_text.insert("1.0", prompt)
            
            # Update navigation
            self.update_history_navigation()
    
    def update_history_navigation(self):
        """Update history navigation buttons and label"""
        total = len(self.process_history)
        current = self.current_history_index + 1 if total > 0 else 0
        
        self.history_label.configure(text=f"{current}/{total}")
        
        # Update button states
        self.prev_button.configure(state="normal" if self.current_history_index > 0 else "disabled")
        self.next_button.configure(state="normal" if self.current_history_index < total - 1 else "disabled")
    
    def add_to_history(self, prompt: str, result: str):
        """Add a new result to history"""
        self.process_history.append((prompt, result))
        self.current_history_index = len(self.process_history) - 1
        self.update_history_navigation()
    
    def check_error_queue(self):
        """Check for errors from background threads"""
        try:
            while True:
                error_type, error_msg = self.error_queue.get_nowait()
                messagebox.showerror(error_type, error_msg)
        except queue.Empty:
            pass
        finally:
            self.after(100, self.check_error_queue)
    
    def save_api_key(self):
        """Save API key for session"""
        api_key = self.api_key_entry.get()
        if api_key:
            self.api_key = api_key
            self.save_api_button.configure(text="✓")
            self.after(2000, lambda: self.save_api_button.configure(text="Save"))
            self.update_status("API key saved")
            self.update_button_states()  # Add this line
    
    def copy_to_clipboard(self):
        """Copy current tab content to clipboard"""
        current_tab = self.tabview.get()
        if "Transcription" in current_tab and self.transcribed_text:
            self.clipboard_clear()
            self.clipboard_append(self.transcribed_text)
            self.update_status("Transcription copied to clipboard")
        elif "Result" in current_tab and self.current_history_index >= 0:
            _, result = self.process_history[self.current_history_index]
            self.clipboard_clear()
            self.clipboard_append(result)
            self.update_status("Result copied to clipboard")
        else:
            self.update_status("No content to copy")
    
    def load_whisper_model(self):
        """Load Whisper model in background thread"""
        if self.model_loading:
            return
            
        def load():
            self.model_loading = True
            self.after(0, lambda: self.update_status("Loading Whisper model..."))
            self.after(0, lambda: self.model_status.configure(text="🟡 Loading..."))
            
            try:
                model_name = self.model_var.get()
                self.whisper_model = whisper.load_model(model_name)
                
                self.after(0, lambda: self.update_status("Whisper model loaded"))
                self.after(0, lambda: self.model_status.configure(text="🟢 Ready"))
                self.after(0, lambda: self.update_button_states())
                
            except Exception as e:
                self.error_queue.put(("Model Load Error", f"Failed to load Whisper model: {str(e)}"))
                self.after(0, lambda: self.model_status.configure(text="🔴 Error"))
            finally:
                self.model_loading = False
        
        threading.Thread(target=load, daemon=True).start()

    def show_network_plot_in_panel(self, plot_path):
        """Display network plot in the visualization panel with dynamic resizing"""
        try:
            from PIL import Image, ImageTk
            
            # Store the plot path for resizing
            self.current_network_plot_path = plot_path
            
            # Load the original image
            img = Image.open(plot_path)
            
            # Get the current size of the network display area
            display_width = self.network_display.winfo_width()
            display_height = self.network_display.winfo_height()
            
            # If display area is not yet rendered, use default sizes
            if display_width <= 1 or display_height <= 1:
                display_width = 500
                display_height = 400
            
            # Calculate scaling to maintain aspect ratio with padding
            padding = 20
            max_width = display_width - padding
            max_height = display_height - padding
            
            img_width, img_height = img.size
            scale = min(max_width/img_width, max_height/img_height)
            
            # Limit maximum scale to avoid pixelation
            scale = min(scale, 2.0)
            
            new_width = int(img_width * scale)
            new_height = int(img_height * scale)
            
            # Resize image with high quality
            img = img.resize((new_width, new_height), Image.Resampling.LANCZOS)
            
            # Convert to PhotoImage
            photo = ImageTk.PhotoImage(img)
            
            # Update the network display label with the image
            self.network_display.configure(image=photo, text="")
            self.network_display.image = photo  # Keep a reference
            self.network_photo = photo  # Store reference for resizing
            
            # Enable refresh button
            if hasattr(self, 'refresh_network_btn'):
                self.refresh_network_btn.configure(state="normal")
            
            # Add or update save button
            if not hasattr(self, 'save_network_btn'):
                self.save_network_btn = ctk.CTkButton(
                    self.network_frame,
                    text="💾 Save Plot",
                    command=lambda: self.save_network_plot(plot_path),
                    height=30
                )
                self.save_network_btn.grid(row=2, column=0, pady=5)
            
        except Exception as e:
            self.error_queue.put(("Display Error", f"Failed to display network plot: {str(e)}"))

    def create_network_plot(self):
        """Create a network plot using Word2Vec embeddings with user-selectable clustering"""
        if not self.transcribed_text:
            messagebox.showwarning("No Content", "No transcription available for network plot")
            return
        
        api_key = self.api_key_entry.get() or self.api_key
        if not api_key:
            messagebox.showwarning("API Key Required", "Please enter your Anthropic API key")
            return
        
        # Ask user for number of clusters
        cluster_dialog = ctk.CTkInputDialog(
            text="Enter number of clusters (2-10):",
            title="Cluster Configuration"
        )
        n_clusters_str = cluster_dialog.get_input()
        
        try:
            n_clusters = int(n_clusters_str) if n_clusters_str else 5
            n_clusters = max(2, min(10, n_clusters))
        except:
            n_clusters = 5
        
        def generate_plot():
            try:
                self.after(0, lambda: self.network_button.configure(state="disabled"))
                self.after(0, lambda: self.update_status("Loading Word2Vec model..."))
                self.after(0, lambda: self.progress_bar.set(0.1))
                
                # Load or cache Word2Vec model with better error handling
                if self.word2vec_model is None:
                    try:
                        import gensim.downloader as api
                        # Try to load a smaller, faster model first
                        self.after(0, lambda: self.update_status("Downloading Word2Vec model (first time only)..."))
                        self.word2vec_model = api.load('glove-wiki-gigaword-50')
                    except Exception as e:
                        self.error_queue.put(("Model Error", f"Failed to load Word2Vec model: {str(e)}\nPlease install: pip install gensim"))
                        return
                
                self.after(0, lambda: self.update_status("Processing text with Word2Vec..."))
                self.after(0, lambda: self.progress_bar.set(0.3))
                
                # Preprocess text
                text = self.transcribed_text.lower()
                words = re.findall(r'\b[a-z]+\b', text)
                
                # Enhanced stop words list
                stop_words = {'the', 'a', 'an', 'and', 'or', 'but', 'in', 'on', 'at', 'to', 'for',
                             'of', 'with', 'by', 'from', 'as', 'is', 'was', 'are', 'were', 'be',
                             'been', 'being', 'have', 'has', 'had', 'do', 'does', 'did', 'will',
                             'would', 'could', 'should', 'may', 'might', 'must', 'shall', 'can',
                             'need', 'dare', 'ought', 'used', 'i', 'you', 'he', 'she', 'it', 'we',
                             'they', 'them', 'their', 'this', 'that', 'these', 'those'}
                
                # Filter words and keep only those in Word2Vec vocabulary
                filtered_words = []
                for w in words:
                    if w not in stop_words and len(w) > 2:
                        try:
                            # Check if word is in model vocabulary
                            if hasattr(self.word2vec_model, 'key_to_index'):
                                if w in self.word2vec_model.key_to_index:
                                    filtered_words.append(w)
                            elif w in self.word2vec_model:
                                filtered_words.append(w)
                        except:
                            pass
                
                if len(filtered_words) < 5:
                    self.error_queue.put(("Insufficient Data", "Not enough words found in the Word2Vec vocabulary. The text may be too short or contain specialized terms."))
                    return
                
                # Get most frequent words that have embeddings
                word_freq = Counter(filtered_words)
                top_words = [word for word, freq in word_freq.most_common(150)]
                
                if len(top_words) < 5:
                    self.error_queue.put(("Insufficient Data", "Not enough words with embeddings found. Try with longer text."))
                    return
                
                # Get Word2Vec embeddings
                embeddings = []
                valid_words = []
                for word in top_words:
                    try:
                        if hasattr(self.word2vec_model, 'get_vector'):
                            embeddings.append(self.word2vec_model.get_vector(word))
                        else:
                            embeddings.append(self.word2vec_model[word])
                        valid_words.append(word)
                    except:
                        pass
                
                if len(embeddings) < 5:
                    self.error_queue.put(("Insufficient Embeddings", "Not enough word embeddings found. The text may be too specialized."))
                    return
                
                embeddings = np.array(embeddings)
                
                self.after(0, lambda: self.update_status("Clustering words in embedding space..."))
                self.after(0, lambda: self.progress_bar.set(0.4))
                
                # Perform clustering in high-dimensional space
                from sklearn.cluster import KMeans
                n_clusters_actual = min(n_clusters, len(valid_words) // 2)
                kmeans = KMeans(n_clusters=n_clusters_actual, random_state=42, n_init=10)
                cluster_labels = kmeans.fit_predict(embeddings)
                
                # Create co-occurrence information for edge weights
                window_size = 7
                co_occurrences = {}
                word_positions = {word: [] for word in valid_words}
                
                # Record positions of valid words
                for i, word in enumerate(filtered_words):
                    if word in valid_words:
                        word_positions[word].append(i)
                
                # Calculate co-occurrences with distance weighting
                for word1 in valid_words:
                    for word2 in valid_words:
                        if word1 != word2:
                            pair = tuple(sorted([word1, word2]))
                            if pair not in co_occurrences:
                                co_occurrences[pair] = 0
                                for pos1 in word_positions[word1]:
                                    for pos2 in word_positions[word2]:
                                        distance = abs(pos1 - pos2)
                                        if distance <= window_size:
                                            # Weight by inverse distance
                                            co_occurrences[pair] += 1.0 / (1 + distance)
                
                self.after(0, lambda: self.update_status("Projecting to 2D space..."))
                self.after(0, lambda: self.progress_bar.set(0.5))
                
                # Project to 2D using t-SNE for better visualization
                from sklearn.manifold import TSNE
                from sklearn.decomposition import PCA
                
                if len(embeddings) > 30:
                    # Use PCA first for dimensionality reduction if many words
                    pca = PCA(n_components=min(30, len(embeddings)-1))
                    embeddings_reduced = pca.fit_transform(embeddings)
                    tsne = TSNE(n_components=2, perplexity=min(30, len(embeddings)-1), 
                               random_state=42, n_iter=1000)
                    coords_2d = tsne.fit_transform(embeddings_reduced)
                else:
                    perplexity = min(5, len(embeddings)-1)  # Ensure perplexity is valid
                    tsne = TSNE(n_components=2, perplexity=perplexity, 
                               random_state=42, n_iter=1000)
                    coords_2d = tsne.fit_transform(embeddings)
                
                # Create network graph
                import networkx as nx
                G = nx.Graph()
                
                # Add nodes with positions and attributes
                for i, word in enumerate(valid_words):
                    G.add_node(word, 
                              pos=(coords_2d[i, 0], coords_2d[i, 1]),
                              cluster=cluster_labels[i],
                              weight=word_freq[word])
                
                # Add edges based only on co-occurrence from the transcript
                for i, word1 in enumerate(valid_words):
                    for j, word2 in enumerate(valid_words[i+1:], i+1):
                        pair = tuple(sorted([word1, word2]))
                        co_occur = co_occurrences.get(pair, 0)
                        
                        # Add edge only if words co-occur in the transcript
                        if co_occur > 0.25:  # Threshold for meaningful co-occurrence
                            G.add_edge(word1, word2, weight=co_occur)
                
                self.after(0, lambda: self.update_status("Creating visualization..."))
                self.after(0, lambda: self.progress_bar.set(0.7))
                
                # Create visualization
                import matplotlib.pyplot as plt
                fig, ax = plt.subplots(figsize=(10, 8))
                fig.patch.set_facecolor('#f0f0f0')
                ax.set_facecolor('#ffffff')
                
                # Use positions from t-SNE (semantic positioning)
                pos = nx.get_node_attributes(G, 'pos')
                
                # Color scheme for clusters
                colors = plt.cm.Set3(np.linspace(0, 1, n_clusters_actual))
                node_colors = [colors[G.nodes[node]['cluster']] for node in G.nodes()]
                
                # Node sizes based on word frequency
                node_sizes = [200 + G.nodes[node]['weight'] * 20 for node in G.nodes()]
                
                # Draw edges with varying thickness based on co-occurrence strength
                edges = G.edges()
                if edges:
                    edge_weights = [G[u][v]['weight'] for u, v in edges]
                    max_weight = max(edge_weights) if edge_weights else 1
                    
                    # Draw edges with thickness proportional to co-occurrence
                    for (u, v), weight in zip(edges, edge_weights):
                        alpha = 0.3 + (weight / max_weight) * 0.5
                        width = 0.5 + (weight / max_weight) * 3
                        nx.draw_networkx_edges(G, pos, [(u, v)], alpha=alpha, 
                                             width=width, edge_color='#666666', ax=ax)
                
                # Draw nodes
                nx.draw_networkx_nodes(G, pos, node_color=node_colors, 
                                     node_size=node_sizes, alpha=0.85,
                                     edgecolors='black', linewidths=1.5, ax=ax)
                
                # Draw labels
                labels = {node: node for node in G.nodes()}
                nx.draw_networkx_labels(G, pos, labels, font_size=8, 
                                       font_weight='bold', ax=ax)
                
                # Add title
                ax.set_title("Semantic Network: Word2Vec Similarity & Co-occurrence", 
                            fontsize=12, fontweight='bold', pad=20)
                ax.axis('off')
                
                # Add cluster legend
                for i in range(n_clusters_actual):
                    cluster_words = [w for w in valid_words if cluster_labels[valid_words.index(w)] == i][:3]
                    label = f"Cluster {i+1}: {', '.join(cluster_words[:3])}..."
                    ax.scatter([], [], c=[colors[i]], s=150, label=label, alpha=0.85)
                
                ax.legend(loc='upper left', frameon=True, fancybox=True, 
                         shadow=True, fontsize=8)
                
                plt.tight_layout()
                
                # Save plot
                plot_path = Path("network_analysis.png")
                plt.savefig(plot_path, dpi=150, bbox_inches='tight', 
                           facecolor='#f0f0f0', edgecolor='none')
                plt.close()
                
                # Display plot in the panel instead of new window
                self.after(0, lambda: self.show_network_plot_in_panel(plot_path))
                self.after(0, lambda: self.update_status("Network plot generated successfully"))
                self.after(0, lambda: self.progress_bar.set(1.0))
                
            except Exception as e:
                self.error_queue.put(("Plot Error", f"Failed to generate network plot: {str(e)}"))
            finally:
                self.after(0, lambda: self.progress_bar.set(0))
                self.after(0, lambda: self.network_button.configure(state="normal"))
                if hasattr(self, 'refresh_network_btn'):
                    self.after(0, lambda: self.refresh_network_btn.configure(state="normal"))
        
        threading.Thread(target=generate_plot, daemon=True).start()
    
    def on_model_change(self, value):
        """Handle model selection change"""
        self.whisper_model = None
        self.model_status.configure(text="⚪ Not loaded")
        self.update_button_states()
        self.load_whisper_model()
    
    def select_audio_file(self):
        """Open file dialog to select audio/video file"""
        file_types = [
            ("Audio/Video files", "*.mp3 *.wav *.m4a *.flac *.aac *.ogg *.wma *.mp4 *.mov *.avi *.mkv *.webm *.m4v"),
            ("Audio files", "*.mp3 *.wav *.m4a *.flac *.aac *.ogg *.wma"),
            ("Video files", "*.mp4 *.mov *.avi *.mkv *.webm *.m4v"),
            ("MP3 files", "*.mp3"),
            ("MP4 files", "*.mp4"),
            ("WAV files", "*.wav"),
            ("All files", "*.*")
        ]
        
        file_path = filedialog.askopenfilename(
            title="Select Audio/Video File",
            filetypes=file_types
        )
        
        if file_path:
            self.audio_file_path = file_path
            filename = os.path.basename(file_path)
            self.file_label.configure(text=f"Selected: {filename}")
            
            # Update valid range for the new file
            self.update_valid_range()
            
            # Reset transcription when new file is loaded
            self.transcribed_text = ""
            self.trans_text.delete("1.0", "end")
            self.trans_word_count.configure(text="Words: 0 | Characters: 0")
            self.trans_status.configure(text="Ready to transcribe", text_color=("gray50", "gray50"))
            
            # Clear network display and semantic summary
            if hasattr(self, 'network_display'):
                self.network_display.configure(
                    text="No network generated yet\n\nClick 'Generate Network Plot' to visualize",
                    image=None
                )
                if hasattr(self.network_display, 'image'):
                    self.network_display.image = None
            
            if hasattr(self, 'semantic_summary_text'):
                self.semantic_summary_text.configure(state="normal")
                self.semantic_summary_text.delete("1.0", "end")
                self.semantic_summary_text.insert("1.0", "No summary generated yet.\n\nClick 'Generate' to create a semantic summary of the transcribed text.")
                self.semantic_summary_text.configure(state="disabled")
            
            # Update button states
            self.update_button_states()
            
            # Determine file type for status message
            file_ext = os.path.splitext(file_path)[1].lower()
            video_extensions = {'.mp4', '.mov', '.avi', '.mkv', '.webm', '.m4v'}
            file_type = "video" if file_ext in video_extensions else "audio"
            self.update_status(f"{file_type.capitalize()} file selected: {filename}")
    
    def update_button_states(self):
        """Update all button states based on current conditions including translation"""
        
        # Get current API key
        current_api_key = self.api_key if self.api_key else self.api_key_entry.get()
        
        # Transcribe button - ALWAYS enable if we have a file AND model is loaded
        if self.audio_file_path and self.whisper_model:
            self.transcribe_button.configure(state="normal", fg_color=("#1f538d", "#14375e"))
        else:
            self.transcribe_button.configure(state="disabled", fg_color=("gray75", "gray25"))
        
        # Clean button - enable if we have transcribed text AND API key
        if self.transcribed_text and current_api_key:
            self.clean_button.configure(state="normal", fg_color=("#1f538d", "#14375e"))
        else:
            self.clean_button.configure(state="disabled", fg_color=("gray75", "gray25"))
        
        # Network button - enable if we have transcribed text AND API key
        if self.transcribed_text and current_api_key:
            self.network_button.configure(state="normal", fg_color=("#1f538d", "#14375e"))
            if hasattr(self, 'refresh_network_btn'):
                self.refresh_network_btn.configure(state="normal")
        else:
            self.network_button.configure(state="disabled", fg_color=("gray75", "gray25"))
            if hasattr(self, 'refresh_network_btn'):
                self.refresh_network_btn.configure(state="disabled")
        
        # Summarize button - enable if we have transcribed text AND API key
        if self.transcribed_text and current_api_key:
            self.summarize_button.configure(state="normal", fg_color=("#1f538d", "#14375e"))
        else:
            self.summarize_button.configure(state="disabled", fg_color=("gray75", "gray25"))
        
        # Translate button - enable if we have transcribed text AND API key
        if hasattr(self, 'translate_button'):
            if self.transcribed_text and current_api_key:
                self.translate_button.configure(state="normal", fg_color=("#1f538d", "#14375e"))
            else:
                self.translate_button.configure(state="disabled", fg_color=("gray75", "gray25"))
        
        # Custom prompt button - enable if we have transcribed text AND API key
        if self.transcribed_text and current_api_key:
            self.process_custom_button.configure(state="normal")
        else:
            self.process_custom_button.configure(state="disabled")
        
        # Semantic summary button - enable if we have transcribed text AND API key
        if self.transcribed_text and current_api_key:
            if hasattr(self, 'generate_summary_btn'):
                self.generate_summary_btn.configure(state="normal")
        else:
            if hasattr(self, 'generate_summary_btn'):
                self.generate_summary_btn.configure(state="disabled")
        
        # Update time segment inputs when duration info changes
        if hasattr(self, 'start_time_entry'):
            self.start_time_entry.bind("<KeyRelease>", lambda e: self.update_duration_info())
            self.end_time_entry.bind("<KeyRelease>", lambda e: self.update_duration_info())

    def generate_semantic_summary(self):
        """Generate ONE sentence that captures BOTH semantics and tone."""
        if not self.transcribed_text:
            messagebox.showwarning("No Content", "No transcription available for summary")
            return
    
        api_key = self.api_key_entry.get() or self.api_key
        if not api_key:
            messagebox.showwarning("API Key Required", "Please enter your Anthropic API key")
            return
    
        def generate():
            try:
                self.after(0, lambda: self.generate_summary_btn.configure(state="disabled"))
                self.after(0, lambda: self.update_status("Generating semantic & tone summary..."))
                self.after(0, lambda: self.progress_bar.set(0.5))
    
                client = Anthropic(api_key=api_key)
                text_to_summarize = self.transcribed_text
    
                prompt = f"""You will receive a transcript. Write EXACTLY ONE sentence (≤ 30 words) that captures BOTH:
    - the core meaning/topic, and
    - the overall tone/affect/delivery style (e.g., enthusiastic, cautious, frustrated, formal).
    
    Rules:
    - Single sentence only; end with a period.
    - ≤ 30 words.
    - No quotes, labels, lists, JSON, or extra commentary.
    - No markdown.
    
    Transcript:
    {text_to_summarize}"""
    
                response = client.messages.create(
                    model="claude-sonnet-4-20250514",
                    max_tokens=150,
                    temperature=0.2,
                    messages=[{"role": "user", "content": prompt}]
                )
    
                raw = response.content[0].text.strip()
    
                # --- Post-process to enforce one sentence and ≤ 30 words ---
                s = raw.replace("\n", " ").strip()
                s = s.strip(' "\'')  # remove leading/trailing quotes if any
    
                import re
                # Keep only the first sentence boundary encountered
                parts = re.split(r'(?<=[.!?])\s+', s)
                s = parts[0].strip() if parts else s
    
                # Ensure it ends with a sentence terminator
                if not s.endswith(('.', '!', '?')):
                    s += '.'
    
                # Enforce ≤ 30 words
                words = s.split()
                if len(words) > 30:
                    s = " ".join(words[:30]).rstrip('.,;:!?') + "."
    
                self.after(0, lambda: self.update_semantic_summary_display(s))
                self.after(0, lambda: self.update_status("Semantic & tone summary generated"))
    
            except Exception as e:
                self.error_queue.put(("Summary Error", f"Failed to generate summary: {str(e)}"))
            finally:
                self.after(0, lambda: self.progress_bar.set(0))
                self.after(0, lambda: self.generate_summary_btn.configure(state="normal"))
    
        threading.Thread(target=generate, daemon=True).start()
    
    def update_semantic_summary_display(self, summary):
        """Update the semantic summary display in the visualization panel"""
        self.semantic_summary_text.configure(state="normal")
        self.semantic_summary_text.delete("1.0", "end")
        self.semantic_summary_text.insert("1.0", summary)
        self.semantic_summary_text.configure(state="disabled")
        
        # Add timestamp
        from datetime import datetime
        timestamp = datetime.now().strftime("%H:%M:%S")
        
        # Update with timestamp info
        self.semantic_summary_text.configure(state="normal")
        self.semantic_summary_text.insert("end", f"\n\n[Generated at {timestamp}]")
        self.semantic_summary_text.configure(state="disabled")
    
    def update_status(self, message: str):
        """Update status label"""
        self.status_label.configure(text=message)
    
    def start_transcription(self):
        """Start transcription in background thread with time segment support"""
        if not self.audio_file_path or not self.whisper_model:
            return
        
        # Validate time segment
        if not self.validate_time_segment():
            return
        
        # Get time segment parameters
        start_time, end_time = self.get_time_segment_params()
        
        def transcribe():
            try:
                # Determine file type for status message
                file_ext = os.path.splitext(self.audio_file_path)[1].lower()
                video_extensions = {'.mp4', '.mov', '.avi', '.mkv', '.webm', '.m4v'}
                file_type = "video" if file_ext in video_extensions else "audio"
                
                # Update status with segment info
                if start_time is not None and end_time is not None:
                    segment_info = f" (segment: {start_time:.1f}s - {end_time:.1f}s)"
                else:
                    segment_info = " (full)"
                
                self.after(0, lambda: self.update_status(f"Transcribing {file_type}{segment_info}..."))
                self.after(0, lambda: self.progress_bar.set(0.1))
                self.after(0, lambda: self.transcribe_button.configure(state="disabled"))
                
                # Prepare transcription parameters
                transcribe_params = {}
                
                # Handle time segments
                if start_time is not None and end_time is not None:
                    # For time segments, we need to extract the segment first
                    import tempfile
                    import subprocess
                    
                    # Create temporary file for segment
                    with tempfile.NamedTemporaryFile(suffix='.wav', delete=False) as temp_file:
                        temp_path = temp_file.name
                    
                    try:
                        # Extract segment using ffmpeg
                        duration = end_time - start_time
                        cmd = [
                            'ffmpeg', '-i', self.audio_file_path,
                            '-ss', str(start_time),
                            '-t', str(duration),
                            '-acodec', 'pcm_s16le',
                            '-ar', '16000',
                            '-ac', '1',
                            '-y',  # Overwrite output file
                            temp_path
                        ]
                        
                        result = subprocess.run(cmd, capture_output=True, text=True, timeout=60)
                        
                        if result.returncode != 0:
                            # If ffmpeg fails, try alternative approach with Whisper's built-in segment handling
                            # This is less accurate but works as fallback
                            self.after(0, lambda: self.update_status("Using alternative segment extraction..."))
                            audio_path = self.audio_file_path
                            
                            # Whisper doesn't have built-in segment support, so we'll process full and trim text
                            # This is a workaround - ideally ffmpeg should work
                            transcribe_params['initial_prompt'] = f"[Transcribe from {start_time:.1f}s to {end_time:.1f}s]"
                        else:
                            audio_path = temp_path
                        
                        # Transcribe the segment
                        result = self.whisper_model.transcribe(audio_path, **transcribe_params)
                        transcribed_text = result["text"]
                        
                    finally:
                        # Clean up temporary file
                        if 'temp_path' in locals() and os.path.exists(temp_path):
                            try:
                                os.remove(temp_path)
                            except:
                                pass
                else:
                    # Transcribe full audio
                    result = self.whisper_model.transcribe(self.audio_file_path, **transcribe_params)
                    transcribed_text = result["text"]
                
                # Update UI on main thread
                self.after(0, lambda: self.update_transcription_result(transcribed_text))
                
                # Store segment info for reference
                self.last_transcription_segment = (start_time, end_time)
                
            except subprocess.TimeoutExpired:
                self.error_queue.put(("Transcription Error", "Segment extraction timed out. Try a shorter segment."))
            except FileNotFoundError:
                self.error_queue.put(("FFmpeg Error", "FFmpeg not found. Please install FFmpeg for time segment support."))
            except Exception as e:
                self.error_queue.put(("Transcription Error", f"Failed to transcribe {file_type}: {str(e)}"))
            finally:
                self.after(0, lambda: self.progress_bar.set(0))
                self.after(0, lambda: self.update_button_states())
        
        threading.Thread(target=transcribe, daemon=True).start()
    
    def update_transcription_result(self, text: str):
        """Update transcription result in UI"""
        self.transcribed_text = text
        self.trans_text.delete("1.0", "end")
        self.trans_text.insert("1.0", text)
        
        # Update word count
        words = len(text.split())
        chars = len(text)
        self.trans_word_count.configure(text=f"Words: {words} | Characters: {chars}")
        self.trans_status.configure(text="Transcription complete", text_color=("green", "lightgreen"))
        
        self.update_status("Transcription completed")
        self.progress_bar.set(1.0)
        self.update_button_states()
    
    def summarize_transcription(self):
        """Summarize transcription using Claude"""
        if not self.transcribed_text:
            return
        
        try:
            word_limit = int(self.word_limit_var.get() or "500")
        except ValueError:
            word_limit = 500
        
        prompt = self.get_summarize_prompt(self.transcribed_text, word_limit)
        
        def process():
            try:
                self.after(0, lambda: self.summarize_button.configure(state="disabled"))
                
                result = self.process_with_claude_enhanced(self.transcribed_text, prompt)
                
                if result:
                    self.after(0, lambda: self.update_result(prompt, result))
                    self.after(0, lambda: self.update_status("Summarization completed"))
                else:
                    self.after(0, lambda: self.update_status("Summarization failed"))
                
            except Exception as e:
                self.error_queue.put(("Processing Error", f"Failed to process: {str(e)}"))
            finally:
                self.after(0, lambda: self.progress_bar.set(0))
                self.after(0, lambda: self.update_button_states())
        
        threading.Thread(target=process, daemon=True).start()
    
    def process_custom_prompt(self):
        """Process custom prompt with Claude"""
        if not self.transcribed_text:
            return
        
        custom_prompt = self.custom_prompt_text.get("1.0", "end-1c")
        
        # Replace {text} placeholder with actual transcription
        prompt = custom_prompt.replace("{text}", self.transcribed_text)
        
        def process():
            try:
                self.after(0, lambda: self.process_custom_button.configure(state="disabled"))
                
                result = self.process_with_claude_enhanced(self.transcribed_text, prompt)
                
                if result:
                    self.after(0, lambda: self.update_result(custom_prompt, result))
                    self.after(0, lambda: self.update_status("Custom prompt processed"))
                else:
                    self.after(0, lambda: self.update_status("Processing failed"))
                
            except Exception as e:
                self.error_queue.put(("Processing Error", f"Failed to process: {str(e)}"))
            finally:
                self.after(0, lambda: self.progress_bar.set(0))
                self.after(0, lambda: self.update_button_states())
        
        threading.Thread(target=process, daemon=True).start()
    
    def update_result(self, prompt: str, result: str):
        """Update result in UI and add to history"""
        self.add_to_history(prompt, result)
        
        self.result_text.delete("1.0", "end")
        self.result_text.insert("1.0", result)
        
        self.result_status.configure(text="Processing complete", text_color=("green", "lightgreen"))
        
        # Switch to result tab
        self.tabview.set("✨ Processed Result")
    
    def get_summarize_prompt(self, text: str, word_limit: int) -> str:
        """Generate summarization prompt with word limit"""
        output_format = self.output_format.get().lower()
        
        format_instructions = {
            "markdown": "Format your response in clean Markdown with appropriate headers and structure.",
            "plain text": "Provide a plain text response without any formatting.",
            "json": "Return your response as a well-structured JSON object.",
            "bullet points": "Structure your response as clear bullet points.",
            "latex pdf": "Format your response in LaTeX document format with proper document class, sections, and formatting."
        }
        
        prompt = f"""Please provide a comprehensive summary of the following transcribed audio content in approximately {word_limit} words. 
        Focus on the main ideas, key points, and important details. 
        {format_instructions.get(output_format, '')}
        
        Transcribed text:
        {text}"""
        
        return prompt
    
    def process_with_claude_enhanced(self, text: str, prompt: str, is_translation: bool = False) -> Optional[str]:
        """Enhanced process with Claude API supporting translations and multiple output formats"""
        api_key = self.api_key_entry.get() or self.api_key
        
        try:
            self.after(0, lambda: self.update_status("Calling Claude API..."))
            self.after(0, lambda: self.progress_bar.set(0.7))
            
            client = Anthropic(api_key=api_key)
            
            # Check if this is a translation and if target language requires special handling
            target_language = None
            if is_translation and hasattr(self, 'target_language_var'):
                target_language = self.target_language_var.get()
            
            # Add instructions for math formatting if LaTeX output is selected
            if self.output_format.get() == "LaTeX PDF":
                if target_language:
                    prompt += f"\n\nIMPORTANT: Format all mathematical equations using proper LaTeX syntax. Use equation environments for display equations and inline math mode for inline expressions. Ensure all special symbols are properly escaped. Include appropriate language packages for {target_language}."
                else:
                    prompt += "\n\nIMPORTANT: Format all mathematical equations using proper LaTeX syntax. Use equation environments for display equations and inline math mode for inline expressions. Ensure all special symbols are properly escaped."
            
            response = client.messages.create(
                model="claude-sonnet-4-20250514",
                max_tokens=4000,
                temperature=0.3,
                messages=[{"role": "user", "content": prompt}]
            )
            
            result_text = response.content[0].text
            
            # Handle LaTeX PDF output if selected
            if self.output_format.get() == "LaTeX PDF":
                if target_language:
                    result_text = self.ensure_latex_format_multilingual(result_text, target_language)
                else:
                    result_text = self.ensure_latex_format(result_text)
            
            self.after(0, lambda: self.progress_bar.set(1.0))
            return result_text
            
        except Exception as e:
            self.error_queue.put(("Claude API Error", f"Failed to process: {str(e)}"))
            return None
    
    def ensure_latex_format(self, text: str) -> str:
        """Ensure text is in proper LaTeX format with beautiful math rendering"""
        
        # Clean up problematic characters
        text = text.replace('\u2019', "'")
        text = text.replace('\u201c', '``')
        text = text.replace('\u201d', "''")
        text = text.replace('\u2013', '--')
        text = text.replace('\u2014', '---')
        
        # Convert markdown-style math to LaTeX if present
        import re
        
        # Convert inline code blocks with math to LaTeX math mode
        text = re.sub(r'```\n?(.*?)\n?```', r'\\[\n\1\n\\]', text, flags=re.DOTALL)
        text = re.sub(r'`([^`]+)`', r'$\1$', text)
        
        # Convert markdown headers to LaTeX sections
        text = re.sub(r'^### (.*?)$', r'\\subsection{\1}', text, flags=re.MULTILINE)
        text = re.sub(r'^## (.*?)$', r'\\section{\1}', text, flags=re.MULTILINE)
        text = re.sub(r'^# (.*?)$', r'\\chapter{\1}', text, flags=re.MULTILINE)
        
        # Convert numbered items to subsections
        text = re.sub(r'^(\d+)\.\s*([A-Z].*?)$', r'\\subsection*{\\textbf{\1. \2}}', text, flags=re.MULTILINE)
        
        # Convert bullet points
        text = re.sub(r'^- (.*?)$', r'\\item \1', text, flags=re.MULTILINE)
        
        # Handle "Where:" sections
        if 'Where:' in text:
            lines = text.split('\n')
            new_lines = []
            in_where = False
            
            for line in lines:
                if 'Where:' in line:
                    new_lines.append(line.replace('Where:', '\\textbf{Where:}'))
                    new_lines.append('\\begin{itemize}')
                    in_where = True
                elif in_where and line.strip() and not line.strip().startswith('\\item'):
                    if line.strip().startswith('-'):
                        new_lines.append(line)
                    elif re.match(r'^\d+\.', line.strip()) or line.strip().startswith('\\'):
                        new_lines.append('\\end{itemize}')
                        new_lines.append(line)
                        in_where = False
                    else:
                        new_lines.append(line)
                else:
                    new_lines.append(line)
            
            if in_where:
                new_lines.append('\\end{itemize}')
            
            text = '\n'.join(new_lines)
        
        # Process mathematical equations
        def format_equations(text):
            """Format equations for proper LaTeX display"""
            lines = text.split('\n')
            formatted_lines = []
            
            for line in lines:
                # Check if line contains an equation pattern
                if '=' in line and any(char in line for char in ['(', ')', '²', '·', 'Σ', '∂', 'φ', 'τ', 'ε']):
                    equation = line.strip()
                    
                    # Replace special characters with LaTeX commands
                    replacements = [
                        ('²', '^2'),
                        ('·', ' \\cdot '),
                        ('Σ', '\\sum'),
                        ('∂', '\\partial'),
                        ('φ', '\\phi'),
                        ('Δφ', '\\Delta\\phi'),
                        ('τ', '\\tau'),
                        ('ε', '\\varepsilon'),
                        ('σ', '\\sigma'),
                        ('→', '\\rightarrow'),
                        ('√', '\\sqrt'),
                    ]
                    
                    for old, new in replacements:
                        equation = equation.replace(old, new)
                    
                    # Format simple fractions
                    equation = re.sub(r'(\w+)\s*/\s*(\w+)', r'\\frac{\1}{\2}', equation)
                    
                    # Wrap in equation environment if not already wrapped
                    if not equation.startswith('\\[') and not equation.startswith('$$'):
                        formatted_lines.append('\\begin{equation}')
                        formatted_lines.append(equation)
                        formatted_lines.append('\\end{equation}')
                    else:
                        formatted_lines.append(equation)
                else:
                    formatted_lines.append(line)
            
            return '\n'.join(formatted_lines)
        
        # Apply equation formatting
        text = format_equations(text)
        
        if not text.startswith("\\documentclass"):
            # Create a complete LaTeX document
            latex_text = r"""\documentclass[11pt]{article}
    \usepackage[utf8]{inputenc}
    \usepackage[T1]{fontenc}
    \usepackage{lmodern}
    \usepackage{amsmath}
    \usepackage{amssymb}
    \usepackage{amsfonts}
    \usepackage{amsthm}
    \usepackage{mathtools}
    \usepackage{geometry}
    \usepackage{enumitem}
    \usepackage{hyperref}
    \geometry{a4paper, margin=1in}
    
    \title{Audio Transcription Analysis}
    \author{Generated Analysis}
    \date{\today}
    
    \begin{document}
    \maketitle
    
    """ + text + r"""
    
    \end{document}"""
            return latex_text
        else:
            # Document already has structure
            if "\\usepackage{amsmath}" not in text:
                lines = text.split('\n')
                for i, line in enumerate(lines):
                    if line.startswith("\\documentclass"):
                        math_packages = [
                            "\\usepackage[utf8]{inputenc}",
                            "\\usepackage[T1]{fontenc}",
                            "\\usepackage{amsmath}",
                            "\\usepackage{amssymb}",
                            "\\usepackage{amsfonts}",
                        ]
                        for j, pkg in enumerate(math_packages):
                            if pkg not in text:
                                lines.insert(i + 1 + j, pkg)
                        break
                text = '\n'.join(lines)
        
        return text

    def clean_text_for_export_enhanced(self, text: str, language: str = None) -> str:
        """Enhanced text cleaning for export with language-specific handling"""
        # Basic cleaning for all languages
        replacements = [
            ('\u200b', ''),  # Zero-width space
            ('\ufeff', ''),  # Zero-width no-break space
        ]
        
        # Language-specific cleaning
        if language and "Chinese" not in language and "Japanese" not in language and "Korean" not in language:
            # Only clean these for non-CJK languages
            replacements.extend([
                ('\u2019', "'"),
                ('\u201c', '"'),
                ('\u201d', '"'),
                ('\u2013', '-'),
                ('\u2014', '--'),
                ('\u2026', '...'),
                ('\u00a0', ' '),
            ])
        
        for old, new in replacements:
            text = text.replace(old, new)
        
        return text

    def get_encoding_for_language(self, language: str) -> str:
        """Get appropriate encoding for different languages"""
        if not language:
            return 'utf-8'
        
        encoding_map = {
            "Chinese (Simplified)": "utf-8",
            "Chinese (Traditional)": "utf-8",
            "Japanese": "utf-8",
            "Korean": "utf-8",
            "Arabic": "utf-8",
            "Hebrew": "utf-8",
            "Russian": "utf-8",
            "Greek": "utf-8",
            "Thai": "utf-8",
            "Hindi": "utf-8",
            "Bengali": "utf-8"
        }
        
        return encoding_map.get(language, 'utf-8')
    
    def export_text_enhanced(self, text_type: str):
        """Enhanced export function supporting multiple languages and formats"""
        if text_type == "transcription" and not self.transcribed_text:
            messagebox.showwarning("No Content", "No transcription to export")
            return
        elif text_type == "result" and (self.current_history_index < 0 or not self.process_history):
            messagebox.showwarning("No Content", "No processed result to export")
            return
        
        output_format = self.output_format.get()
        
        # Check if content is a translation
        is_translation = False
        target_language = None
        if text_type == "result" and self.current_history_index >= 0:
            _, content = self.process_history[self.current_history_index]
            if content.startswith("[Translation to"):
                is_translation = True
                # Extract language from header
                import re
                match = re.search(r'\[Translation to (.+?)\]', content)
                if match:
                    target_language = match.group(1)
        
        if output_format == "LaTeX PDF":
            # For LaTeX PDF, we need to compile to PDF with language support
            self.export_as_latex_pdf_enhanced(text_type, target_language)
        else:
            # Regular text export with proper encoding for different languages
            default_ext = self.get_default_extension()
            
            file_path = filedialog.asksaveasfilename(
                defaultextension=default_ext,
                filetypes=[
                    ("Text Files", "*.txt"),
                    ("Markdown Files", "*.md"),
                    ("JSON Files", "*.json"),
                    ("LaTeX Files", "*.tex"),
                    ("All Files", "*.*")
                ]
            )
            
            if file_path:
                if text_type == "transcription":
                    content = self.transcribed_text
                else:
                    _, content = self.process_history[self.current_history_index]
                
                try:
                    # Determine encoding based on language
                    encoding = self.get_encoding_for_language(target_language)
                    
                    # Use appropriate encoding with error handling
                    with open(file_path, 'w', encoding=encoding, errors='replace') as f:
                        # Clean problematic characters before saving
                        content = self.clean_text_for_export_enhanced(content, target_language)
                        f.write(content)
                    messagebox.showinfo("Export Success", f"File saved successfully")
                except Exception as e:
                    # Fallback to UTF-8 with BOM for better compatibility
                    try:
                        with open(file_path, 'w', encoding='utf-8-sig', errors='replace') as f:
                            f.write(content)
                        messagebox.showinfo("Export Success", f"File saved successfully (UTF-8 with BOM)")
                    except Exception as e2:
                        messagebox.showerror("Export Error", f"Failed to save file: {str(e2)}")
    
    def get_default_extension(self) -> str:
        """Get default file extension based on output format"""
        format_map = {
            "Markdown": ".md",
            "Plain Text": ".txt",
            "JSON": ".json",
            "Bullet Points": ".txt",
            "LaTeX PDF": ".tex"
        }
        return format_map.get(self.output_format.get(), ".txt")
    
    def export_as_latex_pdf_enhanced(self, text_type: str, target_language: str = None):
        """Enhanced LaTeX PDF export with multi-language support"""
        if text_type == "transcription":
            if not self.transcribed_text:
                messagebox.showwarning("No Content", "No transcription to export")
                return
            content = self.transcribed_text
        else:
            if self.current_history_index < 0 or not self.process_history:
                messagebox.showwarning("No Content", "No processed result to export")
                return
            _, content = self.process_history[self.current_history_index]
        
        # Ensure content is in LaTeX format with language support
        if target_language:
            latex_content = self.ensure_latex_format_multilingual(content, target_language)
        else:
            latex_content = self.ensure_latex_format(content)
        
        # Ask user where to save PDF
        pdf_path = filedialog.asksaveasfilename(
            defaultextension=".pdf",
            filetypes=[("PDF Files", "*.pdf"), ("All Files", "*.*")]
        )
        
        if not pdf_path:
            return
        
        try:
            # Create temporary directory for LaTeX compilation
            with tempfile.TemporaryDirectory() as temp_dir:
                tex_file = Path(temp_dir) / "document.tex"
                
                # Write LaTeX content with proper encoding handling
                encoding = 'utf-8' if not target_language else self.get_encoding_for_language(target_language)
                with open(tex_file, 'w', encoding=encoding, errors='replace') as f:
                    f.write(latex_content)
                
                # Try to compile with pdflatex or xelatex for better Unicode support
                try:
                    # Try different possible paths for LaTeX compilers
                    latex_compilers = [
                        ("xelatex", ["/usr/local/texlive/2025/bin/universal-darwin/xelatex",
                                    "/usr/local/texlive/2024/bin/universal-darwin/xelatex",
                                    "/Library/TeX/texbin/xelatex",
                                    "xelatex"]),
                        ("pdflatex", ["/usr/local/texlive/2025/bin/universal-darwin/pdflatex",
                                     "/usr/local/texlive/2024/bin/universal-darwin/pdflatex",
                                     "/Library/TeX/texbin/pdflatex",
                                     "pdflatex"])
                    ]
                    
                    compiler_cmd = None
                    compiler_name = None
                    
                    # Prefer XeLaTeX for better Unicode support
                    if target_language in ["Chinese (Simplified)", "Chinese (Traditional)", 
                                          "Japanese", "Korean", "Arabic", "Hebrew"]:
                        latex_compilers.reverse()  # Try XeLaTeX first for these languages
                    
                    for name, paths in latex_compilers:
                        for path in paths:
                            if os.path.exists(path) or path in ["xelatex", "pdflatex"]:
                                try:
                                    subprocess.run([path, "--version"], capture_output=True, timeout=2)
                                    compiler_cmd = path
                                    compiler_name = name
                                    break
                                except:
                                    continue
                        if compiler_cmd:
                            break
                    
                    if not compiler_cmd:
                        raise FileNotFoundError("No LaTeX compiler found")
                    
                    # Run LaTeX compiler with proper encoding
                    env = os.environ.copy()
                    env['LANG'] = 'en_US.UTF-8'
                    
                    for _ in range(2):
                        result = subprocess.run(
                            [compiler_cmd, "-interaction=nonstopmode", "-output-directory", temp_dir, str(tex_file)],
                            capture_output=True,
                            text=True,
                            timeout=30,
                            env=env,
                            encoding='utf-8',
                            errors='replace'
                        )
                    
                    # Check if PDF was created
                    pdf_file = Path(temp_dir) / "document.pdf"
                    if pdf_file.exists():
                        # Copy to destination
                        import shutil
                        shutil.copy(pdf_file, pdf_path)
                        messagebox.showinfo("Export Success", f"PDF exported successfully using {compiler_name}")
                    else:
                        raise Exception("PDF file was not generated")
                            
                except FileNotFoundError:
                    # LaTeX not installed or not found
                    messagebox.showerror(
                        "LaTeX Not Found",
                        "LaTeX compiler could not be found.\n\n"
                        "Please install MacTeX or TeX Live.\n"
                        "For better Unicode support, XeLaTeX is recommended.\n\n"
                        "Alternatively, you can export as .tex and compile manually."
                    )
                    
                    # Offer to save as .tex instead
                    if messagebox.askyesno("Save as LaTeX", "Would you like to save as a .tex file instead?"):
                        tex_path = pdf_path.replace('.pdf', '.tex')
                        with open(tex_path, 'w', encoding='utf-8', errors='replace') as f:
                            f.write(latex_content)
                        messagebox.showinfo("Export Success", f"LaTeX file saved to {tex_path}")
                            
        except Exception as e:
            messagebox.showerror("Export Error", f"Failed to export PDF: {str(e)}")

    def clean_transcribed_text(self):
        """Clean transcribed text using Claude API"""
        if not self.transcribed_text:
            messagebox.showwarning("No Content", "No transcription to clean")
            return
        
        api_key = self.api_key_entry.get() or self.api_key
        if not api_key:
            messagebox.showwarning("API Key Required", "Please enter your Anthropic API key")
            return
        
        def clean():
            try:
                self.after(0, lambda: self.clean_button.configure(state="disabled"))
                self.after(0, lambda: self.update_status("Cleaning transcription..."))
                self.after(0, lambda: self.progress_bar.set(0.5))
                
                client = Anthropic(api_key=api_key)
                
                prompt = f"""Clean the following transcribed text by:
    1. Correcting any obvious transcription errors or strange words
    2. Fixing grammar and punctuation while preserving the original meaning
    3. Removing filler words (um, uh, etc.) where appropriate
    4. Making the text flow naturally as written prose
    5. DO NOT summarize or remove content - just clean and correct
    
    Return ONLY the cleaned text without any commentary or explanations.
    
    Text to clean:
    {self.transcribed_text}"""
                
                response = client.messages.create(
                    model="claude-sonnet-4-20250514",
                    max_tokens=4000,
                    temperature=0.2,
                    messages=[{"role": "user", "content": prompt}]
                )
                
                cleaned_text = response.content[0].text
                
                # Update transcription with cleaned text
                self.after(0, lambda: self.update_transcription_result(cleaned_text))
                self.after(0, lambda: self.trans_status.configure(
                    text="Transcription cleaned", 
                    text_color=("blue", "lightblue")
                ))
                self.after(0, lambda: self.update_status("Transcription cleaned successfully"))
                
            except Exception as e:
                self.error_queue.put(("Cleaning Error", f"Failed to clean text: {str(e)}"))
            finally:
                self.after(0, lambda: self.progress_bar.set(0))
                self.after(0, lambda: self.clean_button.configure(state="normal"))
        
        threading.Thread(target=clean, daemon=True).start()
    
    # Function to save network plot
    def save_network_plot(self, source_path):
        """Save network plot to user-specified location"""
        file_path = filedialog.asksaveasfilename(
            defaultextension=".png",
            filetypes=[
                ("PNG Files", "*.png"),
                ("All Files", "*.*")
            ]
        )
        
        if file_path:
            import shutil
            try:
                shutil.copy(source_path, file_path)
                messagebox.showinfo("Save Success", "Network plot saved successfully")
            except Exception as e:
                messagebox.showerror("Save Error", f"Failed to save plot: {str(e)}")
    
    def toggle_recording(self):
        """Start or stop audio recording"""
        if not self.is_recording:
            self.start_recording()
        else:
            self.stop_recording()
    
    def start_recording(self):
        """Start recording audio from microphone with fallback options"""
        try:
            # First try with sounddevice (preferred method)
            try:
                import sounddevice as sd
                devices = sd.query_devices()
                input_device = sd.default.device[0]
                
                if input_device is None:
                    raise Exception("No microphone detected with sounddevice")
                
                # Use sounddevice method
                self._start_recording_sounddevice()
                return
                
            except (ImportError, Exception) as e:
                print(f"Sounddevice not available: {e}")
                
                # Fallback to pyaudio if sounddevice fails
                try:
                    import pyaudio
                    self._start_recording_pyaudio()
                    return
                except ImportError:
                    pass
                
                # Final fallback: use system command (macOS/Linux)
                if sys.platform in ['darwin', 'linux']:
                    self._start_recording_system()
                else:
                    messagebox.showerror(
                        "Recording Error", 
                        "Audio recording requires sounddevice or pyaudio.\n"
                        "Please install: pip install sounddevice\n"
                        "If that fails, try: pip install pyaudio"
                    )
                    
        except Exception as e:
            messagebox.showerror("Recording Error", f"Failed to start recording: {str(e)}")
            self.is_recording = False
    
    def _start_recording_sounddevice(self):
        """Recording with sounddevice (original method)"""
        import sounddevice as sd
        
        self.is_recording = True
        self.recording_data = []
        self.recording_start_time = time.time()
        
        # Update UI
        self.record_button.configure(
            text="⏹️ Stop Recording",
            fg_color=("red", "darkred")
        )
        self.recording_status.configure(text="Recording...")
        self.use_recording_button.configure(state="disabled")
        
        # Start recording in background thread
        def record_audio():
            try:
                with sd.InputStream(
                    samplerate=self.recording_samplerate,
                    channels=1,
                    callback=self.audio_callback
                ):
                    while self.is_recording:
                        time.sleep(0.1)
                        # Update timer on main thread
                        elapsed = time.time() - self.recording_start_time
                        mins, secs = divmod(int(elapsed), 60)
                        self.after(0, lambda: self.recording_timer.configure(
                            text=f"⏱️ {mins:02d}:{secs:02d}"
                        ))
            except Exception as e:
                self.error_queue.put(("Recording Error", str(e)))
                self.after(0, self.stop_recording)
        
        self.recording_thread = threading.Thread(target=record_audio, daemon=True)
        self.recording_thread.start()
    
    def _start_recording_pyaudio(self):
        """Fallback recording with pyaudio"""
        import pyaudio
        
        self.is_recording = True
        self.recording_data = []
        self.recording_start_time = time.time()
        
        # Update UI
        self.record_button.configure(
            text="⏹️ Stop Recording",
            fg_color=("red", "darkred")
        )
        self.recording_status.configure(text="Recording (pyaudio)...")
        self.use_recording_button.configure(state="disabled")
        
        def record_audio():
            try:
                p = pyaudio.PyAudio()
                
                stream = p.open(
                    format=pyaudio.paInt16,
                    channels=1,
                    rate=self.recording_samplerate,
                    input=True,
                    frames_per_buffer=1024
                )
                
                while self.is_recording:
                    data = stream.read(1024, exception_on_overflow=False)
                    audio_array = np.frombuffer(data, dtype=np.int16).astype(np.float32) / 32768.0
                    self.recording_data.append(audio_array.reshape(-1, 1))
                    
                    # Update timer
                    elapsed = time.time() - self.recording_start_time
                    mins, secs = divmod(int(elapsed), 60)
                    self.after(0, lambda: self.recording_timer.configure(
                        text=f"⏱️ {mins:02d}:{secs:02d}"
                    ))
                
                stream.stop_stream()
                stream.close()
                p.terminate()
                
            except Exception as e:
                self.error_queue.put(("Recording Error", str(e)))
                self.after(0, self.stop_recording)
        
        self.recording_thread = threading.Thread(target=record_audio, daemon=True)
        self.recording_thread.start()
    
    def _start_recording_system(self):
        """System command fallback for macOS/Linux"""
        self.is_recording = True
        self.recording_start_time = time.time()
        
        # Create temporary file for recording
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        recordings_dir = Path("recordings")
        recordings_dir.mkdir(exist_ok=True)
        self.recorded_file_path = recordings_dir / f"recording_{timestamp}.wav"
        
        # Update UI
        self.record_button.configure(
            text="⏹️ Stop Recording",
            fg_color=("red", "darkred")
        )
        self.recording_status.configure(text="Recording (system)...")
        self.use_recording_button.configure(state="disabled")
        
        def record_audio():
            try:
                if sys.platform == 'darwin':  # macOS
                    # Use sox or ffmpeg if available
                    cmd = [
                        'ffmpeg', '-f', 'avfoundation', '-i', ':0',
                        '-t', '3600',  # Max 1 hour
                        '-ar', str(self.recording_samplerate),
                        '-ac', '1',
                        str(self.recorded_file_path)
                    ]
                else:  # Linux
                    cmd = [
                        'arecord', '-f', 'cd', '-t', 'wav',
                        '-d', '3600',  # Max 1 hour
                        str(self.recorded_file_path)
                    ]
                
                self.recording_process = subprocess.Popen(
                    cmd,
                    stdout=subprocess.PIPE,
                    stderr=subprocess.PIPE
                )
                
                # Update timer while recording
                while self.is_recording and self.recording_process.poll() is None:
                    time.sleep(0.1)
                    elapsed = time.time() - self.recording_start_time
                    mins, secs = divmod(int(elapsed), 60)
                    self.after(0, lambda: self.recording_timer.configure(
                        text=f"⏱️ {mins:02d}:{secs:02d}"
                    ))
                
            except Exception as e:
                self.error_queue.put(("Recording Error", str(e)))
                self.after(0, self.stop_recording)
        
        self.recording_thread = threading.Thread(target=record_audio, daemon=True)
        self.recording_thread.start()
    
    def audio_callback(self, indata, frames, time_info, status):
        """Callback for audio recording"""
        if status:
            print(f"Recording status: {status}")
        self.recording_data.append(indata.copy())
    
    def stop_recording(self):
        """Stop recording and save audio file"""
        if not self.is_recording:
            return
        
        self.is_recording = False
        
        # Update UI
        self.record_button.configure(
            text="🎤 Start Recording",
            fg_color=("gray75", "gray25")
        )
        self.recording_status.configure(text="Processing recording...")
        self.recording_timer.configure(text="")
        
        # Stop system recording if using that method
        if hasattr(self, 'recording_process') and self.recording_process is not None:
            try:
                self.recording_process.terminate()
                self.recording_process.wait(timeout=2)
            except:
                try:
                    self.recording_process.kill()
                except:
                    pass
            
            # For system recording, file is already saved
            if self.recorded_file_path and self.recorded_file_path.exists():
                file_size = self.recorded_file_path.stat().st_size
                if file_size > 0:
                    self.recording_status.configure(
                        text=f"Saved: {self.recorded_file_path.name}"
                    )
                    self.use_recording_button.configure(state="normal")
                    self.update_status(f"Recording saved: {self.recorded_file_path.name}")
                else:
                    self.recording_status.configure(text="Recording failed - empty file")
            
            # Clean up
            self.recording_process = None
            return
        
        # Wait for recording thread to finish (for sounddevice/pyaudio methods)
        if hasattr(self, 'recording_thread') and self.recording_thread:
            self.recording_thread.join(timeout=1)
        
        # Save recording for sounddevice/pyaudio methods
        if self.recording_data:
            try:
                # Combine all audio chunks
                audio_data = np.concatenate(self.recording_data, axis=0)
                
                # Create filename with timestamp
                timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                filename = f"recording_{timestamp}.wav"
                
                # Create recordings directory if it doesn't exist
                recordings_dir = Path("recordings")
                recordings_dir.mkdir(exist_ok=True)
                
                # Save audio file
                self.recorded_file_path = recordings_dir / filename
                
                # Try to save with soundfile, fall back to scipy if needed
                try:
                    import soundfile as sf
                    sf.write(
                        self.recorded_file_path,
                        audio_data,
                        self.recording_samplerate
                    )
                except ImportError:
                    try:
                        # Fallback to scipy
                        from scipy.io import wavfile
                        # Convert float32 to int16 for scipy
                        audio_int16 = (audio_data * 32767).astype(np.int16)
                        wavfile.write(
                            self.recorded_file_path,
                            self.recording_samplerate,
                            audio_int16
                        )
                    except ImportError:
                        # Final fallback: save as numpy array and convert later
                        np.save(str(self.recorded_file_path).replace('.wav', '.npy'), audio_data)
                        messagebox.showwarning(
                            "Format Notice", 
                            "Recording saved as .npy file. Install soundfile or scipy to save as .wav"
                        )
                        filename = filename.replace('.wav', '.npy')
                        self.recorded_file_path = recordings_dir / filename
                
                # Update UI
                duration = len(audio_data) / self.recording_samplerate
                mins, secs = divmod(int(duration), 60)
                self.recording_status.configure(
                    text=f"Saved: {filename} ({mins:02d}:{secs:02d})"
                )
                self.use_recording_button.configure(state="normal")
                self.update_status(f"Recording saved: {filename}")
                
            except Exception as e:
                messagebox.showerror("Save Error", f"Failed to save recording: {str(e)}")
                self.recording_status.configure(text="Failed to save recording")
        else:
            self.recording_status.configure(text="No audio recorded")
        
        # Clean up
        self.recording_data = []
        self.recording_thread = None
    
    def use_recording(self):
        """Use the last recording as input file"""
        if self.recorded_file_path and self.recorded_file_path.exists():
            self.audio_file_path = str(self.recorded_file_path)
            filename = os.path.basename(self.audio_file_path)
            self.file_label.configure(text=f"Selected: {filename}")
            
            # Update valid range for the recording
            self.update_valid_range()
            
            # Reset transcription when new recording is used
            self.transcribed_text = ""
            self.trans_text.delete("1.0", "end")
            self.trans_word_count.configure(text="Words: 0 | Characters: 0")
            self.trans_status.configure(text="Ready to transcribe", text_color=("gray50", "gray50"))
            
            # Clear visualization panel
            if hasattr(self, 'network_display'):
                self.network_display.configure(
                    text="No network generated yet\n\nClick 'Generate Network Plot' to visualize",
                    image=None
                )
                if hasattr(self.network_display, 'image'):
                    self.network_display.image = None
            
            if hasattr(self, 'semantic_summary_text'):
                self.semantic_summary_text.configure(state="normal")
                self.semantic_summary_text.delete("1.0", "end")
                self.semantic_summary_text.insert("1.0", "No summary generated yet.\n\nClick 'Generate' to create a semantic summary of the transcribed text.")
                self.semantic_summary_text.configure(state="disabled")
            
            # Update button states to enable transcribe button
            self.update_button_states()
            self.update_status(f"Using recording: {filename}")
            
            # Switch to transcription tab
            self.tabview.set("📝 Transcription")
        else:
            messagebox.showwarning("No Recording", "No recording available to use")
    
    def clear_all(self):
        """Clear all text content (modified to stop recording if active)"""
        # Stop recording if in progress
        if self.is_recording:
            self.stop_recording()
        
        # Original clear_all code continues...
        if self.transcribed_text or self.process_history:
            if messagebox.askyesno("Clear All", "Clear all content and history?"):
                self.trans_text.delete("1.0", "end")
                self.result_text.delete("1.0", "end")
                self.transcribed_text = ""
                self.process_history = []
                self.current_history_index = -1
                self.trans_word_count.configure(text="Words: 0 | Characters: 0")
                self.trans_status.configure(text="No transcription yet", text_color=("gray50", "gray50"))
                self.result_status.configure(text="No processed result yet", text_color=("gray50", "gray50"))
                self.update_history_navigation()
                self.update_status("Cleared all content")
                self.update_button_states()

def main():
    """Main entry point"""
    # Check for required packages
    required_packages = {
        'whisper': 'openai-whisper',
        'anthropic': 'anthropic',
        'customtkinter': 'customtkinter',
        'sounddevice': 'sounddevice',
        'soundfile': 'soundfile'
    }
    
    missing_packages = []
    for module, package in required_packages.items():
        try:
            __import__(module)
        except ImportError:
            missing_packages.append(package)
    
    if missing_packages:
        print("Missing required packages. Please install them using:")
        print(f"pip install {' '.join(missing_packages)}")
        print("\nFor macOS, you may also need:")
        print("brew install ffmpeg portaudio")  # Added portaudio
        print("\nFor LaTeX PDF export (optional):")
        print("- Windows: Install MiKTeX or TeX Live")
        print("- macOS: Install MacTeX")
        print("- Linux: sudo apt-get install texlive-full")
        sys.exit(1)
    
    # Run the application
    app = AudioAnalyzerApp()
    app.mainloop()

if __name__ == "__main__":
    main()